Dependencies

In [ ]:
import re
from PyPDF2 import PdfReader
import pdfplumber
import tabula
from pdfminer.high_level import extract_text
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import InvalidArgumentException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyautogui
from email.mime.text import MIMEText
import smtplib
from email.mime.multipart import MIMEMultipart
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import logging
import duckdb

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)


Miscellanious Functions

In [ ]:
# Retrieve information from a duckdb file
def get_data(folder,duckdb_file,table_name):
    folder = folder
    duckdb_file = duckdb_file
    db_file = rf"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\{folder}\{duckdb_file}.duckdb"
    # Connect to DuckDB
    con = duckdb.connect(db_file)
    # Current scraped data
    table_name = table_name
    data = con.execute(f"SELECT * FROM {table_name}").fetch_df()
    con.close()
    return data

California

In [ ]:
 # URL of the site
url = 'https://dot.ca.gov/programs/construction/statement-of-ongoing-contracts'

# Start a new browser session
# Automatic driver installer
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url)

EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
year = now.year
month = now.strftime('%B') 
prev_month_date = now - relativedelta(months=1)
prev_month = prev_month_date.strftime('%B')
prev_month_year = prev_month_date.year 

try:
    try:
        current_month_report = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.LINK_TEXT, f'Ongoing Contracts as of {month} {year} Alt Format (XLSX)'))
        )
        current_month_report.click()
        time.sleep(5)
    except TimeoutException:
        print(f"Monthly report for {month} {year} not yet available. Check to confirm.")
        
    previous_month_report = WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.LINK_TEXT, f'Ongoing Contracts as of {prev_month} {prev_month_year} Alt Format (XLSX)'))
    )
    previous_month_report.click()
    time.sleep(5)
    
except TimeoutException:
    print(f"Monthly report for {prev_month} {prev_month_year} not yet available. Check manually to confirm.")
    
finally:
    # Close the browser
    driver.quit()
    


Colorado

In [ ]:
# Extract information from all pages except for page 1

# Regex to match quantities with exactly three decimal places (e.g., "244.000")
qty_re = re.compile(r'^[\d,]+\.\d{3}$')
# General numeric regex (allows any decimal places)
numeric_re = re.compile(r'^-?[\d,]*\.\d+$')

def extract_pdf_to_dataframe(pdf_path):
    records = []
    with pdfplumber.open(pdf_path) as pdf:
        # Skip the very first page
        for page in pdf.pages[1:]:
            text = page.extract_text() or ""
            lines = [ln.strip() for ln in text.split('\n') if ln.strip()]
            i = 0
            while i < len(lines):
                tokens1 = lines[i].split()
                # Identify a line beginning with a 4-digit REF NO
                if len(tokens1) >= 4 and re.match(r'^\d{4}$', tokens1[0]):
                    # Find tokens on line1 that match the 3-decimal pattern
                    qty_idxs = [idx for idx,t in enumerate(tokens1) if qty_re.match(t)]
                    if len(qty_idxs) < 2:
                        i += 1
                        continue

                    # Extract REF, ITEM, CURRENT and PREV quantities
                    ref_no      = tokens1[0]
                    item_no     = tokens1[1]
                    current_qty = tokens1[qty_idxs[-2]]
                    prev_qty    = tokens1[qty_idxs[-1]]

                    # Everything between token 2 and the first qty token is the description
                    desc_parts = tokens1[2: qty_idxs[-2]]
                    desc_lines = [" ".join(desc_parts)]

                    # --- Locate the line with unit_price (Line-3) ---
                    idx_line3 = None
                    for k in range(i+1, len(lines)):
                        first_tok = lines[k].split()[0]
                        if numeric_re.match(first_tok):
                            idx_line3 = k
                            break
                    if idx_line3 is None or idx_line3 - i < 2:
                        i += 1
                        continue
                    idx_line2 = idx_line3 - 1

                    # Merge any wrapped description lines between 1 & 2
                    for k in range(i+1, idx_line2):
                        first_tok = lines[k].split()[0]
                        if not numeric_re.match(first_tok):
                            desc_lines.append(lines[k])
                    full_description = " ".join(desc_lines)

                    # --- Parse Line-2: units, qty this period, amount this period ---
                    t2 = lines[idx_line2].split()
                    num2_idxs = [j for j,t in enumerate(t2) if numeric_re.match(t)]
                    if len(num2_idxs) >= 2:
                        first2 = num2_idxs[0]
                        units       = " ".join(t2[:first2])
                        qty_this    = t2[num2_idxs[0]]
                        amt_this    = t2[num2_idxs[1]]
                    else:
                        units = qty_this = amt_this = ""

                    # --- Parse Line-3: unit_price, qty_to_date, cumulative_amount ---
                    t3 = lines[idx_line3].split()
                    num3 = [tok for tok in t3 if numeric_re.match(tok) or tok.startswith("-")]
                    unit_price        = num3[0] if len(num3)>0 else ""
                    qty_to_date       = num3[1] if len(num3)>1 else ""
                    cumulative_amount = num3[2] if len(num3)>2 else ""

                    # Combine fields
                    current_combined = f"{current_qty} {units} {unit_price}".strip()
                    prev_combined    = f"{prev_qty} {qty_this} {qty_to_date}".strip()

                    records.append({
                        "REF NO.": ref_no,
                        "ITEM NO.": item_no,
                        "ITEM DESCRIPTION": full_description,
                        "CURRENT QUANTITY/UNITS UNIT PRICE": current_combined,
                        "PREV QTY/ QTY THIS PERIOD QTY TO DATE": prev_combined,
                        "AMOUNT THIS PERIOD": amt_this,
                        "CUMULATIVE AMOUNT": cumulative_amount
                    })

                    # Advance past this block
                    i = idx_line3 + 1
                else:
                    i += 1

    return pd.DataFrame(records)

if __name__ == "__main__":
    file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Colorado\Example_11582_27.pdf"
    df = extract_pdf_to_dataframe(file)
    


In [ ]:
# Extract information from page 1 only
# --- CONFIG ---
single_fields = [
    "CONTID", "PCN", "ESTIMATE NO", "SPEC YR", "COFRS REPORTING CATEGORY",
    "FACS REF NO", "CONTRACT DESCRIPTION", "TIME CHARGED", "TIME ALLOW",
    "PERCENT TIME", "ORIG TIME ALLOW", "PROJECT NO", "NAME OF ROAD",
    "PROJECT COUNTIES", "PAY PERIOD ENDING", "DATE TIME STARTED",
    "DATE LET", "DATE WORK BEGAN", "DATE AWARDED", "DATE TIME STOPPED",
    "DATE CONTRACT EXECUTED", "DATE ACCEPTED", "DATE NOTICE TO PROCEED",
    "CURRENT PROJECT AMT", "AWARD PROJECT AMT", "PERCENT COMPLETE",
    "FUNDS AVAILABLE", "TOTAL CLAIMS", "APPROVED FOR PAYMENT BY",
    "PAYMENT DUE", "PROJECT COMMENT", "ESTIMATE COMMENT"
]
two_col_fields = [
    "PARTICIPATING", "NON-PARTICIPATING", "TOTAL EARNINGS",
    "STOCKPILED MATERIALS", "GROSS EARNINGS", "RETAINAGE",
    "SECURITIES ENCUMBERED", "NET EARNINGS", "LIQUIDATED DAMAGES",
    "AUTOPAY ADJUSTMENT", "AMOUNT DUE", "OTHER ADJUSTMENTS"
]
expanded_two_col = [f"{f}_{suf}" for f in two_col_fields for suf in ("CURRENT_TOTAL","THIS_ESTIMATE")]
ALL_COLUMNS = single_fields + expanded_two_col

def extract_page1(pdf_path: str) -> pd.DataFrame:
    # helper to avoid None.group errors
    def safe_search(pattern, txt):
        m = re.search(pattern, txt)
        return m.group(1) if m else ""
    
    # read page1
    with pdfplumber.open(pdf_path) as pdf:
        text = pdf.pages[0].extract_text()
    lines = text.split('\n')
    
    # prepare empty row
    row = {col: "" for col in ALL_COLUMNS}
    
    # --- SINGLE FIELDS ---
    l3 = lines[3] if len(lines)>3 else ""
    row["CONTID"]  = safe_search(r"CONTID:(\S+)", l3)
    row["ESTIMATE NO"] = safe_search(r"ESTIMATE NO:(\S+)", l3)
    row["SPEC YR"] = safe_search(r"FINL SPEC YR:(\d+)", l3)
    
    l4 = lines[4] if len(lines)>4 else ""
    row["PCN"]      = safe_search(r"PCN:(\S+)", l4)
    row["COFRS REPORTING CATEGORY"] = safe_search(r"COFRS REPORTING CATEGORY:(\S+)", l4)
    row["FACS REF NO"] = safe_search(r"FACS REF NO:(\S*)", l4)
    
    l5 = lines[5] if len(lines)>5 else ""
    tc = safe_search(r"TIME CHARGED:\s*([\d,]+\.?\d*)\s*DAYS", l5)
    row["TIME CHARGED"] = f"{tc} DAYS" if tc else ""
    row["TIME ALLOW"]   = safe_search(r"TIME ALLOW:\s*([\d,]+\.?\d* WORK DAYS)", l5)
    
    l6 = lines[6] if len(lines)>6 else ""
    row["CONTRACT DESCRIPTION"] = l6.split("PERCENT TIME")[0].replace("CONTRACT DESCRIPTION:","").strip()
    row["PERCENT TIME"]       = safe_search(r"PERCENT TIME\s*[:]\s*([\d,]+\.?\d*%?)", l6)
    row["ORIG TIME ALLOW"]    = safe_search(r"ORIG TIME ALLOW\s*[:]\s*([\d,]+\.?\d* WORK DAYS)", l6)
    
    l7 = lines[7] if len(lines)>7 else ""
    row["PROJECT NO"]     = safe_search(r"PROJECT NO:\s*(.+)", l7).strip()
    
    l9 = lines[9] if len(lines)>9 else ""
    row["NAME OF ROAD"]   = safe_search(r"NAME OF ROAD:\s*(.+)", l9).strip()
    
    l10 = lines[10] if len(lines)>10 else ""
    row["PROJECT COUNTIES"] = safe_search(r"PROJECT COUNTIES:\s*(.+)", l10).strip()
    
    l12 = lines[12] if len(lines)>12 else ""
    row["PAY PERIOD ENDING"]    = safe_search(r"PAY PERIOD ENDING\s+(\S+)", l12)
    row["DATE TIME STARTED"]    = safe_search(r"DATE TIME STARTED\s+(\S+)", l12)
    
    l13 = lines[13] if len(lines)>13 else ""
    row["DATE LET"]         = safe_search(r"DATE LET\s+(\S+)", l13)
    row["DATE WORK BEGAN"]  = safe_search(r"DATE WORK BEGAN\s+(\S+)", l13)
    
    l14 = lines[14] if len(lines)>14 else ""
    row["DATE AWARDED"]      = safe_search(r"DATE AWARDED\s+(\S+)", l14)
    row["DATE TIME STOPPED"] = safe_search(r"DATE TIME STOPPED\s+(\S+)", l14)
    
    l15 = lines[15] if len(lines)>15 else ""
    row["DATE CONTRACT EXECUTED"] = safe_search(r"DATE CONTRACT EXECUTED\s+(\S+)", l15)
    row["DATE ACCEPTED"]         = safe_search(r"DATE ACCEPTED\s+(\S+)", l15)
    
    l16 = lines[16] if len(lines)>16 else ""
    row["DATE NOTICE TO PROCEED"] = safe_search(r"DATE NOTICE TO PROCEED\s+(\S+)", l16)
    
    # --- AMOUNTS & BRACKETS ---
    l18 = lines[18] if len(lines)>18 else ""
    row["CURRENT PROJECT AMT"] = safe_search(r"CURRENT PROJECT AMT:\s*\$?\s*([\d,]+\.\d+)", l18)
    part = re.search(r"PARTICIPATING\s*\$?\s*([\d,]+\.\d+)\s*\$?\s*(-?[\d,]+\.\d+)", l18)
    if part:
        row["PARTICIPATING_CURRENT_TOTAL"], row["PARTICIPATING_THIS_ESTIMATE"] = part.group(1), part.group(2)
    
    l19 = lines[19] if len(lines)>19 else ""
    row["AWARD PROJECT AMT"] = safe_search(r"AWARD PROJECT AMT:\s*\$?\s*([\d,]+\.\d+)", l19)
    nonp = re.search(r"NON-PARTICIPATING\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l19)
    if nonp:
        row["NON-PARTICIPATING_CURRENT_TOTAL"], row["NON-PARTICIPATING_THIS_ESTIMATE"] = nonp.group(1), nonp.group(2)
    
    l20 = lines[20] if len(lines)>20 else ""
    pc = safe_search(r"PERCENT COMPLETE:\s*([\d,]+\.\d+)%?", l20)
    row["PERCENT COMPLETE"] = f"{pc}%" if pc and not pc.endswith('%') else pc
    te = re.search(r"TOTAL EARNINGS\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l20)
    if te:
        row["TOTAL EARNINGS_CURRENT_TOTAL"], row["TOTAL EARNINGS_THIS_ESTIMATE"] = te.group(1), te.group(2)
    
    l21 = lines[21] if len(lines)>21 else ""
    row["FUNDS AVAILABLE"] = safe_search(r"FUNDS AVAILABLE:\s*\$?\s*([\d,]+\.\d+)", l21)
    sm = re.search(r"STOCKPILED MATERIALS\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l21)
    if sm:
        row["STOCKPILED MATERIALS_CURRENT_TOTAL"], row["STOCKPILED MATERIALS_THIS_ESTIMATE"] = sm.group(1), sm.group(2)
    
    l22 = lines[22] if len(lines)>22 else ""
    ge = re.search(r"GROSS EARNINGS\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l22)
    if ge:
        row["GROSS EARNINGS_CURRENT_TOTAL"], row["GROSS EARNINGS_THIS_ESTIMATE"] = ge.group(1), ge.group(2)
    
    l23 = lines[23] if len(lines)>23 else ""
    row["TOTAL CLAIMS"] = safe_search(r"TOTAL CLAIMS:\s*\$?\s*([\d,]+\.\d+)", l23)
    rt = re.search(r"RETAINAGE\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l23)
    if rt:
        row["RETAINAGE_CURRENT_TOTAL"], row["RETAINAGE_THIS_ESTIMATE"] = rt.group(1), rt.group(2)
    
    l24 = lines[24] if len(lines)>24 else ""
    se = re.search(r"SECURITIES ENCUMBERED\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l24)
    if se:
        row["SECURITIES ENCUMBERED_CURRENT_TOTAL"], row["SECURITIES ENCUMBERED_THIS_ESTIMATE"] = se.group(1), se.group(2)
    
    l25 = lines[25] if len(lines)>25 else ""
    ne = re.search(r"NET EARNINGS\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l25)
    if ne:
        row["NET EARNINGS_CURRENT_TOTAL"], row["NET EARNINGS_THIS_ESTIMATE"] = ne.group(1), ne.group(2)
    
    l26 = lines[26] if len(lines)>26 else ""
    ld = re.search(r"LIQUIDATED DAMAGES\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l26)
    if ld:
        row["LIQUIDATED DAMAGES_CURRENT_TOTAL"], row["LIQUIDATED DAMAGES_THIS_ESTIMATE"] = ld.group(1), ld.group(2)
    
    l27 = lines[27] if len(lines)>27 else ""
    aa = re.search(r"AUTOPAY ADJUSTMENT\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l27)
    if aa:
        row["AUTOPAY ADJUSTMENT_CURRENT_TOTAL"], row["AUTOPAY ADJUSTMENT_THIS_ESTIMATE"] = aa.group(1), aa.group(2)
    
    l28 = lines[28] if len(lines)>28 else ""
    ad = re.search(r"AMOUNT DUE\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l28)
    if ad:
        row["AMOUNT DUE_CURRENT_TOTAL"], row["AMOUNT DUE_THIS_ESTIMATE"] = ad.group(1), ad.group(2)
    
    l29 = lines[29] if len(lines)>29 else ""
    oa = re.search(r"OTHER ADJUSTMENTS\s+(-?[\d,]+\.\d+)\s+(-?[\d,]+\.\d+)", l29)
    if oa:
        row["OTHER ADJUSTMENTS_CURRENT_TOTAL"], row["OTHER ADJUSTMENTS_THIS_ESTIMATE"] = oa.group(1), oa.group(2)
    
    # PAYMENT DUE 
    l30 = lines[30] if len(lines) > 30 else ""
    row["PAYMENT DUE"] = safe_search(r"PAYMENT DUE\s+\$?\s*(-?[\d,]+\.\d+)", l30)
    
    l31 = lines[31] if len(lines) > 31 else ""
    row["APPROVED FOR PAYMENT BY"] = safe_search(r"APPROVED FOR PAYMENT BY(\S*)", l31)

    l32 = lines[32] if len(lines) > 32 else ""
    row["PROJECT COMMENT"] = safe_search(r"PROJECT COMMENT:(\S*)", l32)

    l33 = lines[33] if len(lines) > 33 else ""
    row["ESTIMATE COMMENT"] = safe_search(r"ESTIMATE COMMENT:(\S*)", l33)


    return pd.DataFrame([row], columns=ALL_COLUMNS)

# Example usage:
if __name__ == "__main__":
    file_page1 = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Colorado\Example_11582_27.pdf"
    df1 = extract_page1(file_page1)


WV Pipeline 

In [ ]:
import logging
import duckdb
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import schedule
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
import pytz
def west_virginia_scraping():

    # Configuração de Logging
    logging.basicConfig(
        filename='west_virginia_scraping.log',
        level=logging.INFO,
        format='%(asctime)s:%(levelname)s:%(message)s',
        # handlers=[
        # # logging.FileHandler(r'C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\West_virginia\west_virginia_scraping.log'),  # Log to file
        # logging.StreamHandler() # Log to console
        # ]
    )
    # Email configuration
    SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
    SMTP_PORT = 587  # Usually 587 for TLS
    EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
    EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
    EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

    def send_email(subject, body):
        try:
            msg = MIMEMultipart()
            msg['From'] = EMAIL_SENDER
            msg['To'] = EMAIL_RECIPIENT
            msg['Subject'] = subject
            msg.attach(MIMEText(body, 'plain'))

            with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
                server.starttls()
                server.login(EMAIL_SENDER, EMAIL_PASSWORD)
                server.send_message(msg)
            print("Email notification sent successfully.")
            logging.info("Email notification sent successfully.")
        except Exception as e:
            print(f"Failed to send email notification: {e}")
            logging.error(f"Failed to send email notification: {e}")

    # Automatic driver installer
    service = Service(ChromeDriverManager().install())
    url = 'https://www.wva.state.wv.us/wvdot/surety/'

    # Inicia uma nova sessão do navegador
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    row_data_list_wv = []
    header_data_wv = []

    try:
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'ddlVendors'))
        )
        vendor_dropdown = driver.find_element(By.ID, 'ddlVendors')
        time.sleep(3)
        select = Select(vendor_dropdown)

        for i in range(len(select.options)):
            vendor_dropdown = driver.find_element(By.ID, 'ddlVendors')
            select = Select(vendor_dropdown)
            vendor_name = select.options[i].get_attribute(
                "textContent").strip()
            if i == 0:
                continue  # Skip the 'select vendor' option

            logging.info(f"Scraping for vendor: {vendor_name}")
            select.select_by_visible_text(vendor_name)

            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, '//table[@ID="gvContracts"]/tbody/tr'))
            )

            if i == 1 or not header_data_wv:
                table_header = driver.find_elements(
                    By.XPATH, '//table[@ID="gvContracts"]/tbody/tr/th')
                header_data_wv.extend([header.get_attribute('textContent').strip(
                ) for header in table_header if header.get_attribute('textContent').strip()])
                header_data_wv = ['Contract_Vendors'] + header_data_wv

            table_data = driver.find_elements(
                By.XPATH, '//table[@ID="gvContracts"]/tbody/tr')
            for j, row in enumerate(table_data):
                if j > 0:
                    cells = row.find_elements(By.XPATH, './td')
                    current_row_data = [vendor_name]
                    current_row_data.extend(
                        [cell.text for cell in cells if cell.text])
                    row_data_list_wv.append(current_row_data)

    except TimeoutException as e:
        logging.error(f"Timeout Exception: {e}")
    finally:
        # Fechar o navegador
        driver.quit()

    # Create a Dataframe
    WV_DOT_Data = pd.DataFrame(data=row_data_list_wv, columns=header_data_wv)

    # POST PROCESSING
    df = WV_DOT_Data.copy()
    df.rename(columns = {'Federal/State Project Number':'Project_Number','Description':'Project_Description','Contract Amount':'Project_Cost_Total','Plan Completion Date':'Project_Comp_Est','Substantial Completion Date':'Project_Comp_Substantial',
            'Complete Date':'Project_Comp_Final', 'Release Date': 'Project_Close','Contract_Vendors':'Contractor_Name','Paid Amount':'Payment_Amount_Total','Percent Complete':'Payment_Total_Percent'}, inplace=True)
    def parse_money(value):
        # Remove dollar signs and commas
        value = value.replace('$', '').replace(',', '')
        # Convert values in parentheses to negative numbers
        if '(' in value and ')' in value:
            value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
        return float(value)
    df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
    df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
    df['Payment_Balance'] = df['Project_Cost_Total'] - df['Payment_Amount_Total']
    df['Project_Comp_Est'] = pd.to_datetime(df['Project_Comp_Est'],errors='coerce')
    df['Project_Comp_Substantial'] = pd.to_datetime(df['Project_Comp_Substantial'],errors='coerce')
    df['Project_Comp_Final'] = pd.to_datetime(df['Project_Comp_Final'],errors='coerce')
    df['Project_Close'] = pd.to_datetime(df['Project_Close'],errors='coerce')

    
    df['Project_Comp_Est'] = df['Project_Comp_Est'].dt.strftime('%m/%d/%Y')
    df['Project_Comp_Substantial'] = df['Project_Comp_Substantial'].dt.strftime('%m/%d/%Y')
    df['Project_Comp_Final'] = df['Project_Comp_Final'].dt.strftime('%m/%d/%Y')
    df['Project_Close'] = df['Project_Close'].dt.strftime('%m/%d/%Y')
    
    EST = pytz.timezone('US/Eastern')
    now = datetime.now(EST)
    current_date = now.strftime("%m/%d/%Y")
    df["Pull_Date_Initial"] = current_date

    # DUCKDB INTEGRATION
    # File to store DuckDB data
    db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\West_virginia\data_store_WV.duckdb"
    table_name = "WV_DOT"
    
    # Current scraped data
    scraped_data = df

    # Connect to DuckDB
    con = duckdb.connect(db_file)

    # Create table if not exists
    con.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
    Contractor_Name  TEXT,
    Project_Number  TEXT,
    Payment_Number  INTEGER,
    Project_Description	TEXT,
    Project_Comp_Est  TEXT,
    Project_Comp_Substantial  TEXT,	
    Project_Comp_Final  TEXT,	
    Project_Close  TEXT,
    Payment_Amount DOUBLE,	
    Previous_Payment_Amount_Total DOUBLE,
    Project_Cost_Total  DOUBLE,	
    Payment_Amount_Total  DOUBLE,
    Payment_Total_Percent  TEXT,
    Payment_Balance  DOUBLE,
    Pull_Date_Initial TEXT,
    Payment_Amount_Percent FLOAT,
    )
    """)

    # Insert or Update Logic
    # Load existing data from DuckDB
    existing_data = con.execute(f"SELECT * FROM {table_name}").df()

    # Deduplicate and merge
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
        # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
        combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial', 'Previous_Payment_Amount_Total', 'Payment_Number', 'Payment_Amount_Percent', 'Payment_Amount'])].columns,keep="first") 
        # Post processing before loading into duckdb
        combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
        combined_data = combined_data.sort_values(by=["Project_Number", "Pull_Date_Initial"], ascending=[True,False])
        # Group by Project_Number and calculate the previous Payment_Amount_Total
        combined_data['Previous_Payment_Amount_Total'] = combined_data.groupby("Project_Number")["Payment_Amount_Total"].shift(-1)
        # Assign a reversed payment estimate number
        combined_data["Payment_Number"] = combined_data.groupby("Project_Number").cumcount(ascending=False)
        # Calculate the Current Amount Paid
        combined_data['Payment_Amount'] = combined_data['Payment_Amount_Total'] - combined_data['Previous_Payment_Amount_Total']
        # Fill NaN for the first record in each group (no previous record exists)
        combined_data['Payment_Amount'] = combined_data['Payment_Amount'].fillna(combined_data['Payment_Amount_Total'])
        # Revert the formatting of pull_date_initial column
        combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
        # Calculate Payment Amount Percent
        combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
        table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
        correct_order = table_info['column_name'].tolist()
        # Reorder the DataFrame to avoid conversion errors
        combined_data = combined_data[correct_order]

    else:
        combined_data = scraped_data
        # Post processing before loading into duckdb
        combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
        combined_data = combined_data.sort_values(by=["Project_Number", "Pull_Date_Initial"], ascending=[True,False])
        # Group by Project_Number and calculate the previous Payment_Amount_Total
        combined_data['Previous_Payment_Amount_Total'] = combined_data.groupby("Project_Number")["Payment_Amount_Total"].shift(-1)
        # Assign a reversed payment estimate number
        combined_data["Payment_Number"] = combined_data.groupby("Project_Number").cumcount(ascending=False)
        # Calculate the Current Amount Paid
        combined_data['Payment_Amount'] = combined_data['Payment_Amount_Total'] - combined_data['Previous_Payment_Amount_Total']
        # Fill NaN for the first record in each group (no previous record exists)
        combined_data['Payment_Amount'] = combined_data['Payment_Amount'].fillna(combined_data['Payment_Amount_Total'])
        # Revert the formatting of pull_date_initial column
        combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
        # Calculate Payment Amount Percent
        combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
        table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
        correct_order = table_info['column_name'].tolist()
        # Reorder the DataFrame to avoid conversion errors
        combined_data = combined_data[correct_order]

    # Replace the table with the updated data
    print(combined_data)
    con.execute(f"DELETE FROM {table_name}")
    con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

    # Close connection
    con.close()
    send_email("Python Script Execution Successful", "The job was executed successfully.")
    print("West Virginia scraping completed and DUCKDB file updated Successfully.")
    logging.info(
        'West Virginia scraping completed and DUCKDB file updated Successfully.')
west_virginia_scraping()
# # Function to check the date
# def check_date():
#     EST = pytz.timezone('US/Eastern')
#     now = datetime.now(EST)
#     if now.day in [2, 3]:
#         print(f"Today is {now.strftime('%B %d')}. Starting time checks...")
#         logging.info(f"Today is {now.strftime('%B %d')}. Starting time checks...")
#         check_time_till_execution()

# # Function to check the time every minute on the due date
# def check_time_till_execution():
#     EST = pytz.timezone('US/Eastern')
#     end_time = datetime.now(EST).replace(hour=23, minute=59, second=59)  # End of the day
#     while datetime.now(EST) < end_time:
#         now = datetime.now(EST)
#         if now.hour == 19 and now.minute == 48:
#             print("Scheduler started. Waiting for scheduled jobs...")
#             west_virginia_scraping()
#             break
#         time.sleep(60)  # Check every minute
#     print("Finished time checks for the day.")
#     logging.info("Finished time checks for the day.")

# # Schedule the daily date check
# schedule.every().day.at('19:47').do(check_date) 
# logging.info("Scheduler started. Waiting for scheduled jobs...")
# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("West_Virginia", "data_store_WV", "WV_DOT")
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\West_virginia\Monthly\wv_bulk_pipeline_april_2025_updated.xlsx")

NY City Pipeline 

In [ ]:
import logging
import duckdb
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
import pytz
from openai import OpenAI
import os

def ny_city_scraping():
    # Set download directory
    download_dir = r"C:\Users\TarunPongulaty\Downloads"  # Replace with desired path

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,  # Set default download directory
        "download.prompt_for_download": False,  # Disable download prompts
        "directory_upgrade": True,  # Automatically overwrite files
        "safebrowsing.enabled": True  # Enable safe browsing
    })
    # Automatic driver installer
    service = Service(ChromeDriverManager().install())
    # URL of the site
    url = 'https://wwe2.osc.state.ny.us/transparency/contracts/contractsearch.cfm'
     # Email configuration
    SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
    SMTP_PORT = 587  # Usually 587 for TLS
    EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
    EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
    EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"
    def send_email(subject, body):
            try:
                msg = MIMEMultipart()
                msg['From'] = EMAIL_SENDER
                msg['To'] = EMAIL_RECIPIENT
                msg['Subject'] = subject
                msg.attach(MIMEText(body, 'plain'))

                with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
                    server.starttls()
                    server.login(EMAIL_SENDER, EMAIL_PASSWORD)
                    server.send_message(msg)
                print("Email notification sent successfully.")
                logging.info("Email notification sent successfully.")
            except Exception as e:
                print(f"Failed to send email notification: {e}")
                logging.error(f"Failed to send email notification: {e}")
    # Retreive list of all the state agencies
    # Replace with the correct path to your chromedriver
    def get_agency():
        # Start a new browser session
        driver = webdriver.Chrome(service=service)
        driver.get(url)

        try:
            # wait for agency option to appear
            agency_name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div/form/div[1]/p[1]/span/span[1]/span/ul/li/input"))
            )
            agency_name.click()
            agency_name_dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div/form/div[1]/p[1]/select"))
            )
            select = Select(agency_name_dropdown)
            agency_name_list = [name.get_attribute("textContent").strip() for name in select.options if name.get_attribute("textContent").strip()]  
            # delete "all state agencies" value
            del agency_name_list[0]
            agency_data = pd.DataFrame(data = agency_name_list, columns = ["Agency_Name"])

        finally:
            # Close the browser
            driver.quit()
        return agency_data

    def get_bulk_file():
        try:
            # Start a new browser session
            driver = webdriver.Chrome(service=service, options=chrome_options)
            def is_download_complete(file_path):
                """
                Check if the file is complete by:
                1. Ensuring it's not a .crdownload file (Chrome's in-progress download extension).
                2. Ensuring it has a non-zero size.
                """
                return os.path.exists(file_path) and not file_path.endswith(".crdownload") and os.path.getsize(file_path) > 0

            def wait_for_download(download_dir, timeout=30):
                """
                Wait for the most recently downloaded file to appear in the download directory.
                Ensures the file is fully downloaded before returning its path.
                :param download_dir: The directory to monitor for downloads.
                :param timeout: Maximum time to wait for a file (in seconds).
                :return: The full path of the downloaded file.
                """
                start_time = time.time()
                while True:
                    # List all files in the download directory
                    files = os.listdir(download_dir)

                    if files:
                        # Identify the most recently created file in the directory
                        latest_file = max(files, key=lambda f: os.path.getctime(os.path.join(download_dir, f)))
                        latest_file_path = os.path.join(download_dir, latest_file)

                        # Check if the download is complete
                        if is_download_complete(latest_file_path):
                            return latest_file_path

                    # Break the loop if timeout is exceeded
                    if time.time() - start_time > timeout:
                        raise TimeoutError("Download did not complete within the timeout period.")
                    
                    time.sleep(1)  # Wait for a short time before checking again

            driver.get(url)
            # wait for agency option to appear
            agency_name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div/form/div[1]/p[1]/span/span[1]/span/ul/li/input"))
            )
            agency_name.click()
            # select "all state agencies"
            all_state_agencies = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div[2]/div/form/div[1]/p[1]/select/option[1]"))
            )
            all_state_agencies.click()
            search = driver.find_element(By.ID, 'b')
            # Find the contract number input box and enter the contract number
            search.click()
            # find and download the final report
            download_report = WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.LINK_TEXT, 'Download Summary Contract Information to an Excel Spreadsheet'))
            )
            download_report.click()
            print("Waiting for contract data download to complete...")
            # wait until download is completed
            time.sleep(70)
            try:
                downloaded_file = wait_for_download(download_dir, timeout=60)
                print(f"File successfully downloaded at: {downloaded_file}")
                ny_bulk = pd.read_excel(downloaded_file,skiprows=7)
            except TimeoutError as e:
                print(e)
            download_report_amendment = WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.LINK_TEXT, 'Download Additional Contract and Related Amendment Data for OSC approved transactions'))
            )
            download_report_amendment.click()
            print("Waiting for amendment contract data download to complete...")
            # wait until download is completed
            time.sleep(120)
            try:
                downloaded_file = wait_for_download(download_dir, timeout=60)
                print(f"File successfully downloaded at: {downloaded_file}")
                ny_amendment = pd.read_csv(downloaded_file,skiprows=1)
            except TimeoutError as e:
                print(e)
            
        finally:
            # Close the browser
            driver.quit()
        return ny_bulk, ny_amendment
    
    def append_lookup(unique_departments, agency_data, df_lookup):
        unique_departments = ny_bulk['DEPARTMENT/FACILITY'].unique()
        unique_departments = pd.DataFrame(data=unique_departments,columns=["DEPARTMENT/FACILITY"])
        
        client = OpenAI(api_key=os.getenv("OPEN_API_KEY"))
        agency_data = agency_data
        departments = unique_departments.copy()
        results_df = pd.DataFrame()
        # unique_departments.iloc[i:i+batch_size]
        def get_completion_from_messages(messages, model="gpt-4-turbo", temperature=0):
            response = client.chat.completions.create(
                model=model,
                messages=messages,         
                temperature=temperature, # this is the degree of randomness of the model's output
            )
            return response.choices[0].message.content
        batch_size = 5
        for i in range(0, len(departments), batch_size):
            prompt = f"""
            You are a professional analyst in the department of transportation domain for new york city, an automated service and you should be able to map the department/facility name to its respective Agency. Below are the columns from the two different tables.
            1. 'DEPARTMENT/FACILITY' from 'departments' table
            2. 'Agency_Name' from 'agency_data' table
            To give you more context, a common trend you'd see is that the names of 'Department/Facility' are often similar to their respective 'Agency_Name'. However, in some cases, this is very different and not obvious. So, in such cases,
            please browse the web to find relevant information which can help you map the data. One of the good resources is the following site 'https://wwe2.osc.state.ny.us/TRANSPARENCY/CONTRACTS/CONTRACTSEARCH.CFM'. You can browse other web sources as well to find the relevent information.
            Also, each 'Agency_Name' can have one ore more departments or facilities. So, it is a one to many relationship.
            The following are the 2 tables: {unique_departments.iloc[i:i+batch_size],agency_data}
            """

            context = [ {'role':'system', 'content':prompt} ]# accumulate messages

            messages =  context.copy()
            messages.append(
            {'role':'system', 'content':'''The consolidated table after mapping 'DEPARTMENT/FACILITY' with 'Agency_Name' should exactly be in the form of a json with columns/keys: "Agency_Name", "DEPARTMENT/FACILITY".\
            Also I just want the json output so I'll be able to convert it to a dataframe. Do not add any fillers/words like 'json' around the output.
            Do not skip anything and bypass the token limit if needed'''}) 

            response = get_completion_from_messages(messages, temperature=0)
            # print(response)
            try:
                df_response = pd.read_json(response) 
                results_df = pd.concat([results_df,df_response],ignore_index=True)
            except ValueError:
                print(response)
                print("ERROR AT BATCH ITERATION NUMBER:", i)
                continue 
        # Append changes back to original lookup table
        results_df = pd.concat([df_lookup,results_df],ignore_index=True)
        # Write the updated data to a new lookup table for further review
        results_df.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\Monthly\agency_lookup_table_ny_1.xlsx")
    agency_data = get_agency()
    # Read the latest download bulk files.
    ny_bulk, ny_amendment = get_bulk_file()
    # Read the lookup table
    df_lookup = pd.read_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\agency_lookup_table_ny.xlsx")
    # Check for changes in departments and Agency names
    unique_departments = ny_bulk['DEPARTMENT/FACILITY'].unique()
    unique_departments = pd.DataFrame(data=unique_departments,columns=["DEPARTMENT/FACILITY"])
    previous_unique_departments = df_lookup['DEPARTMENT/FACILITY'].unique()
    previous_unique_departments = pd.DataFrame(data=previous_unique_departments,columns=["DEPARTMENT/FACILITY"])

    departments_join = pd.merge(unique_departments,previous_unique_departments, left_on='DEPARTMENT/FACILITY', right_on = 'DEPARTMENT/FACILITY', how = 'outer', indicator=True)
    departments_join_left_only = departments_join[departments_join['_merge'] == 'left_only']
    previous_unique_agencies = df_lookup['Agency_Name'].unique()
    previous_unique_agencies = pd.DataFrame(data=previous_unique_agencies,columns=["Agency_Name"])
    df_lookup_join = pd.merge(agency_data,previous_unique_agencies, left_on='Agency_Name', right_on = 'Agency_Name', how = 'outer', indicator=True )
    df_lookup_join_left_only = df_lookup_join[df_lookup_join['_merge'] == 'left_only']

    if not departments_join_left_only.empty or not df_lookup_join_left_only.empty:
        send_email('Agency/department Data Change', 'Check if updated Agency/Department data has been written to a new lookup table and review the changes and update the original lookup table')
        append_lookup(departments_join_left_only, agency_data, df_lookup)
        # Reading this new data without reviewing is risky. After review, new agency/department information can be appended easily. To review, compare the new table with the original one to look at differences.
        # df_lookup = pd.read_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\Monthly\agency_lookup_table_ny_1.xlsx") 

    # POST PROCESSING
    # Split into contract type and contract sub type
    ny_bulk[['Contract_Type', 'Contract_Subtype']] = ny_bulk['CONTRACT TYPE'].str.split(' - ', n=1, expand = True)
    del ny_bulk['CONTRACT TYPE'] # use this option if it needs to be deleted
    ny_dot_data = pd.merge(ny_bulk,df_lookup, left_on="DEPARTMENT/FACILITY", right_on="DEPARTMENT/FACILITY", how="left")
    df = ny_dot_data.copy()
    

    df = ny_dot_data.copy()
    df.rename(columns = {'VENDOR NAME':'Contractor_Name','DEPARTMENT/FACILITY': 'Project_Dept_Facility','CONTRACT NUMBER':'Contract_Number','CURRENT CONTRACT AMOUNT':'Project_Cost_Total',
                        'SPENDING TO DATE': 'Payment_Amount_Total',	'CONTRACT START DATE':'Project_Start_Actual','CONTRACT END DATE':'Project_Comp_Est','CONTRACT DESCRIPTION':'Project_Description',
                        'ORIGINAL CONTRACT APPROVED/FILED DATE':'Project_Bid_Awarded'}, inplace=True)
    def parse_money(value):
        # Check if the value is already a float or None/NaN
        if pd.isna(value) or isinstance(value, (float, int)):
            return value
        # Remove dollar signs and commas
        value = value.replace('$', '').replace(',', '')
        # Convert values in parentheses to negative numbers
        if '(' in value and ')' in value:
            value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
        return float(value)
    df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
    df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
    df['Payment_Balance'] = df['Project_Cost_Total'] - df['Payment_Amount_Total']

    EST = pytz.timezone('US/Eastern')
    now = datetime.now(EST)
    current_date = now.strftime("%m/%d/%Y")
    df["Pull_Date_Initial"] = current_date

    # DUCKDB INTEGRATION
    # File to store DuckDB data
    db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\data_store_NY.duckdb"
    table_name = "NY_DOT"

    # Current scraped data
    scraped_data = df

    # Connect to DuckDB
    con = duckdb.connect(db_file)

    # Create table if not exists
    con.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
    Contractor_Name  TEXT,
    Project_Dept_Facility TEXT,
    Agency_Name  TEXT,
    Contract_Number  TEXT,
    Contract_Type  TEXT,
    Contract_Subtype TEXT,
    Payment_Number  INTEGER,
    Project_Description	TEXT,
    Project_Start_Actual  TEXT,
    Project_Comp_Est  TEXT,	
    Project_Bid_Awarded  TEXT,
    Payment_Amount DOUBLE,	
    Previous_Payment_Amount_Total DOUBLE,
    Project_Cost_Total  DOUBLE,	
    Payment_Amount_Total  DOUBLE,
    Payment_Total_Percent  TEXT,
    Payment_Balance  DOUBLE,
    Pull_Date_Initial TEXT,
    Payment_Amount_Percent FLOAT,
    )
    """)

    # Insert or Update Logic
    # Load existing data from DuckDB
    existing_data = con.execute(f"SELECT * FROM {table_name}").df()

    # Deduplicate and merge
    if not existing_data.empty:
        combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
        # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
        combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial','CONTRACT TYPE', 'Contract_Subtype','Payment_Total_Percent','Previous_Payment_Amount_Total', 'Payment_Number', 'Payment_Amount_Percent', 'Payment_Amount'])].columns,keep="first") 
        # Post processing before loading into duckdb
        combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
        combined_data = combined_data.sort_values(by=["Contract_Number", "Pull_Date_Initial"], ascending=[True,False])
        # Group by Contract_Number, Contractor_Name and Project_Description (Since Ny has different jobs within the same contract number) and calculate the previous Payment_Amount_Total
        combined_data['Previous_Payment_Amount_Total'] = combined_data.groupby(["Contract_Number","Contractor_Name","Project_Description"])["Payment_Amount_Total"].shift(-1)
        # Assign a reversed payment estimate number
        combined_data["Payment_Number"] = combined_data.groupby(["Contract_Number","Contractor_Name","Project_Description"]).cumcount(ascending=False)
        # Calculate the Current Amount Paid
        combined_data['Payment_Amount'] = combined_data['Payment_Amount_Total'] - combined_data['Previous_Payment_Amount_Total']
        # Fill NaN for the first record in each group (no previous record exists)
        combined_data['Payment_Amount'] = combined_data['Payment_Amount'].fillna(combined_data['Payment_Amount_Total'])
        # Revert the formatting of pull_date_initial column
        combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
        # Calculate Payment Amount Percent and Payment Total Percent
        combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
        combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
        table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
        correct_order = table_info['column_name'].tolist()
        # Reorder the DataFrame to avoid conversion errors
        combined_data = combined_data[correct_order]

    else:
        print("I am here")
        combined_data = scraped_data
        # Post processing before loading into duckdb
        combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
        combined_data = combined_data.sort_values(by=["Contract_Number", "Pull_Date_Initial"], ascending=[True,False])
        # Group by Contract_Number, Contractor_Name and Project_Description (Since Ny has different jobs within the same contract number) and calculate the previous Payment_Amount_Total
        combined_data['Previous_Payment_Amount_Total'] = combined_data.groupby(["Contract_Number","Contractor_Name","Project_Description"])["Payment_Amount_Total"].shift(-1)
        # Assign a reversed payment estimate number
        combined_data["Payment_Number"] = combined_data.groupby(["Contract_Number","Contractor_Name","Project_Description"]).cumcount(ascending=False)
        # Calculate the Current Amount Paid
        combined_data['Payment_Amount'] = combined_data['Payment_Amount_Total'] - combined_data['Previous_Payment_Amount_Total']
        # Fill NaN for the first record in each group (no previous record exists)
        combined_data['Payment_Amount'] = combined_data['Payment_Amount'].fillna(combined_data['Payment_Amount_Total'])
        # Revert the formatting of pull_date_initial column
        combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
        # Calculate Payment Amount Percent and Payment Total Percent
        combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
        combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
        table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
        correct_order = table_info['column_name'].tolist()
        # Reorder the DataFrame to avoid conersion errors
        combined_data = combined_data[correct_order]

    # Replace the table with the updated data
    con.execute(f"DELETE FROM {table_name}")
    con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

    # Close connection
    con.close()
    send_email("Python Script Execution Successful", "The job was executed successfully.")
    print("NY City scraping completed and DUCKDB file updated Successfully.")
    logging.info(
        'NY City scraping completed and DUCKDB file updated Successfully.')
    ny_amendment.rename(columns = {'TRANSACTION TYPE': 'Transaction_Type','VENDOR NAME':'Contractor_Name','DEPARTMENT/FACILITY': 'Project_Dept_Facility','CONTRACT NUMBER':'Contract_Number','TRANSACTION AMOUNT':'Transaction_Amount',
                        'START DATE':'Start_Date','END DATE':'Project_Comp_Substantial','DESCRIPTION':'Description',
                        'TRANSACTION APPROVED/FILED DATE':'Transaction_Approved/Filed_Date'}, inplace=True)
    ny_amendment.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\Monthly\ny_amendment.xlsx")
ny_city_scraping()


In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("NY_City", "data_store_NY", "NY_DOT")
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\NY_City\Monthly\ny_bulk_pipeline_april.xlsx")

Georgia Pipeline

In [ ]:
# Automatic driver installer
service = Service(ChromeDriverManager().install())

# URL of the site
url = 'https://www.dot.ga.gov/GDOT/pages/contractors.aspx'

# Set Chrome options to disable alerts
chrome_options = Options() 
chrome_options.add_argument("--disable-popup-blocking") 
chrome_options.add_argument("--disable-notifications") 
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-web-security") 
chrome_options.add_argument("--ignore-certificate-errors")

# Contract number to search for
# contract_numbers = list_ga[0:1000] # try with smaller list and clean '#' 
contract_numbers = ['B10007-97-M00-1','APER40-09-127-0'] #contract_id
# contract_numbers = ['0000184.E3000', '0000301', '0000304']  #project_number
# contract_numbers = ['0000078','0000088','0000083']  #PI_number   
primary_counties = ['ALL COUNTIES']  # when value = all counties it is equivalent to the bulk option by primary counties
vendor_names = ['BAKER INFRASTRUCTURE GROUP, INC.']      
header_data_ga = ["contract_id", "description", "source", "pi_numbers", "last_voucher_processed", "project_numbers", "current_contract_amount", "voucher/est_num", "voucher/est_date", "payment_percent_complete","date_approved","net_payment","date_received","balance",
                  "date_processed","material_allowance","date_sent_to_accounting", "total_payments", "estimate_link"]
row_data_list_ga = []
# Create a WebDriver instance with Chrome options
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(url)
original_window = driver.current_window_handle
pi_number = False
contract_id = False
project_number = False
primary_county = True
vendor_name = False
vendor_name_bulk = False # this option may trigger dos attack and lock the account. do not use yet
try: #unexpected alerts need to be handled
    
    # Wait for psi_report to be present
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//button[@class="nb-expando__toggle js-expando__toggle"]'))
    )
    psi_report = driver.find_elements(By.XPATH, '//button[@class="nb-expando__toggle js-expando__toggle"]/div[@class="nb-expando__toggle-text"]')
    for j, row in enumerate(psi_report):
        if j == 4:
            row.click()
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.LINK_TEXT, "PSI"))
    )
    # Navigate to payment dashboard
    psi_link = driver.find_element(By.LINK_TEXT, "PSI")
    psi_link.click()
    # change the window of the driver since new tab is opened
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    # Navigate to payment window dashboard
    payment_info = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="dashboard_page_2_tab"]/tbody/tr/td/div'))
    )
    payment_info.click()

    # Retrive list of all primary counties
    primary_county_dropdown = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
        )
    primary_county_dropdown.click()
    primary_county_labels = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[9]/div/div[2]/div/div/label'))
        ) 
    primary_county_list = [name.get_attribute('textContent').strip() for name in primary_county_labels if name.get_attribute('textContent').strip()]
    
    del primary_county_list[0]
    primary_county_list = primary_county_list
    primary_county_dropdown.click() # disengage the dropdown
    # list to store individual dataframes
    dataframes_payment = []
    time.sleep(5)
    def remove_selection():
        remove_all_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="idShuttleContainerBody"]/tr[3]/td[2]/table/tbody/tr[5]/td'))
            ) 
        remove_all_button.click()

    def check_conditions():
        if contract_id:
            # Find the contract ID dropdown
            # Note: shortcut XPATH doesn't seem to be working because of the HTML structure. So full path is taken
            contract_id_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, "/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[4]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img"))
            )
            contract_id_dropdown.click()
        if pi_number:
            pi_num_dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            pi_num_dropdown.click()
            deselect_default = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[9]/div/div[2]/div[1]/div/label'))
            ) 
            deselect_default.click() 
        if project_number:
            project_num_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[5]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            project_num_dropdown.click()
        if primary_county:
            primary_county_dropdown = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
                )
            primary_county_dropdown.click()
        if vendor_name or vendor_name_bulk:
            vendor_name_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[1]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            vendor_name_dropdown.click()
    def payment_info_selected():
        if vendor_name_bulk:
            move_all_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/table/tbody[1]/tr/td/div[2]/div/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/img'))
            )
            for i in range(10):
                move_all_button.click()

        ok_button_2 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/table/tbody[1]/tr/td/div[3]/a[1]'))
        )
        ok_button_2.click()
        # Deselect "default project" value of 'PI Number' if not already deselected (that is in the case where the search is not by pi_number or primary county or vendor_name)
        # Note: Again in this case shortcut XPATH isn't working so full path is used
        if not pi_number and not primary_county and not vendor_name and not vendor_name_bulk:
            ### This logic needs to be edited to account for when contract ID is true so that "deselect_default" is not triggered more than once (recommended : create a separate def for this logic)    
            pi_num_dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            pi_num_dropdown.click()
            deselect_default = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/div[2]/div[1]/div/label'))
            ) 
            deselect_default.click()
        # Apply the filters
        time.sleep(10)
        apply = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="gobtn"]'))
        )

        apply.click()
        time.sleep(10)
        try:
            #switch the driver to iframe ID
            iframe_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="d:dashboard~p:9297np0cm5nna3pq~x:m7no0s394ckllasi"]'))
            )
            driver.switch_to.frame(iframe_element)
            # format the report to html for interaction
            format = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/table/tbody/tr/td/div/div/div/div/div[2]/div[1]/div/div[2]/a[1]/img[2]'))
            )
            format.click()
            html = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="_xdoFMenu1"]/div/div/ul/li[1]/div/a/div[2]'))
            )
            html.click()
            #switch the driver to the next iframe ID
            time.sleep(15)
            iframe_element_html = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="xdo:docframe0"]'))
            )
            driver.switch_to.frame(iframe_element_html)

            # iterate and extract relevent data for all 
            # driver.switch_to.default_content()
            tables = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '/html/body/table[@class="c29"]'))
            )

            for i, row in enumerate(tables):
                current_data = []
                cells = row.find_elements(By.XPATH,'./tbody/tr/td/p/span')
                siblings = row.find_elements(By.XPATH, './following-sibling::*')
                cost_tables = []
                counter = 0
                for value in cells: 
                    if counter % 2 != 0:
                        current_data.append(value.text)
                    counter +=1
                if len(current_data) != 7:
                    print("check data of type1 table, iteration", i)
                for sibling in siblings:
                    if not project_number and "c39" in sibling.get_attribute("class"):
                        cost_tables.append(sibling)
                    elif project_number and "c41" in sibling.get_attribute("class"):
                        cost_tables.append(sibling)
                    elif "c29" in sibling.get_attribute("class"):
                        break
                for j, cost_table in enumerate(cost_tables):
                    current_cost = []
                    if not project_number:
                        data_table = cost_table.find_element(By.XPATH,'./tbody/tr/td/table[@class="c37"]')
                        data = data_table.find_elements(By.XPATH,'./tbody/tr/td')
                    elif project_number:
                        data_table = cost_table.find_element(By.XPATH,'./tbody/tr/td/table[@class="c39"]')
                        data = data_table.find_elements(By.XPATH,'./tbody/tr/td')
                    counter = 0
                    for cell in data:
                        link_element = cell.find_element(By.TAG_NAME, "a") if len(cell.find_elements(By.TAG_NAME, "a")) > 0 else None
                        if counter % 2 != 0:
                            if link_element:
                                # Get the hyperlink (href attribute)
                                link = link_element.get_attribute("href")
                                current_cost.append(link)
                            else:
                                current_cost.append(cell.text)
                        counter +=1
                    if len(current_cost) != 12:
                        print("check data of type2 table, with following iterations of type1 and type2", i, j)
                    current_cost_join = current_data + current_cost
                    row_data_list_ga.extend([current_cost_join])
            current_df = pd.DataFrame(data=row_data_list_ga, columns = header_data_ga)
            dataframes_payment.append(current_df)
            # switch the driver back to default html content
            driver.switch_to.default_content()
        except TimeoutException:
            print("Data not found for current input of contract numbers.")


    # Enter all contract_id's/vendor names/primary counties
    if not primary_county and not vendor_name and not vendor_name_bulk:
        for i in range(0,len(contract_numbers),100):
            check_conditions() # corresponding dropdown is engaged
            if i == 0:
                search_more = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[9]/div/div[3]/span'))
                    )
            else:
                search_more = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/div[3]/span')) # search button path changes slightly after default is deselected which happens automatically by the second iteration
                    )
            search_more.click()
            remove_selection() # removes selected filters from the previous iteration
            edit_button = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="idShuttleContainerBody"]/tr[1]/th[5]/a/img'))
                )
            edit_button.click() # opens a text box for entering required filters
            contract_id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
            )
            value = contract_numbers[i:i + 100]
            value = "\n".join(value) # Create new lines for every value
            contract_id_input.send_keys(value)
            # county_input.send_keys(Keys.RETURN) # useful only if sending one value at a time that is to create a new line
            ok_button_1 = driver.find_element(By.XPATH, '/html/body/div[12]/div/table/tbody[1]/tr/td/div[3]/a[1]')
            ok_button_1.click() # Button to confirm values for filters after entering in the text box
            payment_info_selected() # Retrieves the data from the selected filters and stores it in a dataframe
        GA_DOT_payment_data = pd.concat(dataframes, ignore_index=True)
    elif primary_county:
        for i in range(0,len(primary_counties),1): # iterate through the list of primary counties that was intially fetched in steps of 20 (adjust as suitable)
            check_conditions() # corresponding dropdown is engaged
            # Navigate to Text area box for entering all the contract ID's, primary counties or any other filters
            if i == 0:
                search_more = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[9]/div/div[3]/span'))
                    )
            else:
                search_more = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/div[3]/span')) # search button path changes slightly after default is deselected which happens automatically by the second iteration
                    )
            search_more.click()
            remove_selection() # removes selected filters from the previous iteration
            edit_button = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="idShuttleContainerBody"]/tr[1]/th[5]/a/img'))
                )
            edit_button.click() # opens a text box for entering required filters
            county_input = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
                )
            value = primary_counties[i:i + 1]
            value = "\n".join(value) # Create new lines for every value
            county_input.send_keys(value)
            # county_input.send_keys(Keys.RETURN) # useful only if sending one value at a time that is to create a new line
            ok_button_1 = driver.find_element(By.XPATH, '/html/body/div[12]/div/table/tbody[1]/tr/td/div[3]/a[1]')
            ok_button_1.click() # Button to confirm values for filters after entering in the text box
            payment_info_selected() # Retrieves the data from the selected filters and stores it in a dataframe
        GA_DOT_payment_data = pd.concat(dataframes_payment, ignore_index=True)

    elif vendor_name: # no need to develop this functionality now, unless required in future
        vendor_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
        )
        for value in vendor_names: 
            vendor_input.send_keys(value)
            vendor_input.send_keys(Keys.RETURN)
            payment_info_selected()

    # Fetch milestone dates information
    dataframes_milestone = []
    milestone_dates = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="dashboard_page_0_tab"]/tbody/tr/td/div'))
    )
    milestone_dates.click()
    def check_conditions1():
        if contract_id:
            # Find the contract ID dropdown
            # Note: shortcut XPATH doesn't seem to be working because of the HTML structure. So full path is taken
            contract_id_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, "/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[4]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img"))
            )
            contract_id_dropdown.click()
        if pi_number:
            pi_num_dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            pi_num_dropdown.click()
            deselect_default = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[9]/div/div[2]/div[1]/div/label'))
            ) 
            deselect_default.click() 
        if project_number:
            project_num_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[5]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            project_num_dropdown.click()
        if primary_county:
            primary_county_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            primary_county_dropdown.click()
        if vendor_name or vendor_name_bulk:
            vendor_name_dropdown = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[1]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            vendor_name_dropdown.click()
    def milestone_data_selected():
        if vendor_name_bulk:
            move_all_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/table/tbody[1]/tr/td/div[2]/div/table/tbody/tr[3]/td[2]/table/tbody/tr[2]/td/img'))
            )
            for i in range(10):
                move_all_button.click()

        ok_button_2 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/table/tbody[1]/tr/td/div[3]/a[1]'))
        )
        ok_button_2.click()
        # Deselect "default project" value of 'PI Number' if not already deselected (that is in the case where the search is not by pi_number or primary county or vendor_name)
        # Note: Again in this case shortcut XPATH isn't working so full path is used
        if not pi_number and not primary_county and not vendor_name and not vendor_name_bulk:    
            pi_num_dropdown = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/table[1]/tbody/tr/td[2]/div/table[1]/tbody/tr/td[2]/div[1]/table/tbody/tr/td[1]/div/table/tbody/tr[2]/td/div/table/tbody/tr/td/div/table/tbody/tr/td/div/div/div/table/tbody/tr/td/div/form/div/table/tbody/tr[2]/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table/tbody/tr/td/div/div[1]/img'))
            )
            pi_num_dropdown.click()
            deselect_default = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/div[2]/div[1]/div/label'))
            ) 
            deselect_default.click()
        # Apply the filters
        time.sleep(10)
        apply = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="gobtn"]'))
        )

        apply.click()
        time.sleep(10)
        #switch the driver to iframe ID
        iframe_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="d:dashboard~p:16a6r3787gg7lutu~x:93gsu676msc1raim"]'))
        )
        driver.switch_to.frame(iframe_element)
        # format the report to html for interaction
        format = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/table/tbody/tr/td/div/div/div/div/div[2]/div[1]/div/div[2]/a[1]/img[2]'))
        )
        format.click()
        html = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="_xdoFMenu1"]/div/div/ul/li[1]/div/a/div[2]'))
        )
        html.click()
        #switch the driver to the next iframe ID
        time.sleep(10)
        iframe_element_next = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="xdo:docframe0"]'))
        )
        driver.switch_to.frame(iframe_element_next)

        # iterate and extract relevent data for all 
        # driver.switch_to.default_content()
        tables = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/table[@class="c7"]'))
        )

        records = []

        # Iterate through the tables in groups of 3
        for i in range(0, len(tables), 3):
            group_tables = tables[i:i+3]
            if len(group_tables) < 3:
                continue  # Skip incomplete groups

            # Extract the vendor name from the first table
            vendor_name = None
            try:
                vendor_name = group_tables[0].find_element(By.CLASS_NAME, "c12").text.strip()
            except:
                pass
            
            # Initialize a dictionary to store the combined data for this vendor
            record = {'Vendor Name': vendor_name}
            # Extract data from the second and third tables
            for data_table in group_tables[1:]:
                rows = data_table.find_elements(By.TAG_NAME, "tr")
            
                for row in rows:
                    cells = row.find_elements(By.TAG_NAME, "td")
                    for idx in range(0, len(cells), 2):
                        try:
                            header = cells[idx].text.strip()
                            value = cells[idx + 1].text.strip() if idx + 1 < len(cells) else None
                            record[header] = value
                        except IndexError:
                            continue  # Skip incomplete rows

            # Add the record to the list
            records.extend([record])
        records_df = pd.DataFrame(records)
        dataframes_milestone.append(records_df) # Append the current iteration dataframe to the list "dataframes1"
        # switch the driver back to default html content
        driver.switch_to.default_content()
    
    # Enter all contract_id's/vendor names/primary counties
    if not primary_county and not vendor_name and not vendor_name_bulk:
        for i in range(0,len(contract_numbers),100):
            check_conditions1() # corresponding dropdown is engaged
            # Navigate to Text area box for entering all the contract ID's
            search_more = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[10]/div/div[3]/span'))
                )
            search_more.click()
            remove_selection()
            edit_button = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="idShuttleContainerBody"]/tr[1]/th[5]/a/img'))
                )
            edit_button.click()
            county_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
            )
            value = primary_counties[i:i + 100]
            value = "\n".join(value) # Create new lines for every value
            county_input.send_keys(value)
            ok_button_1 = driver.find_element(By.XPATH, '/html/body/div[12]/div/table/tbody[1]/tr/td/div[3]/a[1]')
            ok_button_1.click()
            milestone_data_selected()
        GA_DOT_milestone_data = pd.concat(dataframes_milestone, ignore_index=True)
    elif primary_county:
        for i in range(0,len(primary_counties),1): # iterate through the list of primary counties that was intially fetched in steps of 20
            check_conditions1() # corresponding dropdown is engaged
            # Navigate to Text area box for entering all the contract ID's
            search_more = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[9]/div/div[3]/span'))
                )
            search_more.click()
            remove_selection()
            edit_button = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, '//*[@id="idShuttleContainerBody"]/tr[1]/th[5]/a/img'))
                )
            edit_button.click()
            county_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
            )
            value = primary_counties[i:i + 1]
            value = "\n".join(value) # Create new lines for every value
            county_input.send_keys(value)
            ok_button_1 = driver.find_element(By.XPATH, '/html/body/div[12]/div/table/tbody[1]/tr/td/div[3]/a[1]')
            ok_button_1.click()
            milestone_data_selected()
        GA_DOT_milestone_data = pd.concat(dataframes_milestone, ignore_index=True)
    elif vendor_name: # no need to develop this functionality now, unless required in future
        vendor_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="shuttleSelectEditTextArea"]'))
        )
        for value in vendor_names: 
            vendor_input.send_keys(value)
            vendor_input.send_keys(Keys.RETURN)

 
finally:
    # Close the browser
    driver.quit()



In [ ]:
# join payment info data and milestone dates data
GA_DOT_data = pd.merge(GA_DOT_payment_data,GA_DOT_milestone_data, left_on="contract_id", right_on="ContractID:", how='inner')

In [ ]:
# POST PROCESSING
df = GA_DOT_data.copy()
df.rename(columns = {'contract_id':'Contract_Number','project_numbers':'Project_Number','description':'Project_Description','Let Date:':'Project_Letting','current_contract_amount':'Project_Cost_Total','Original Contract Completion Date:':'Project_Comp_Est',
                     'Current Specified Completion Date:':'Project_Comp_Substantial','source': 'Contract_Source','voucher/est_num':'Payment_Number','net_payment':'Payment_Amount','voucher/est_date':'Payment_Date','total_payments':'Payment_Amount_Total',
                     'payment_percent_complete':'Payment_Total_Percent','date_approved':'Payment_Approved','date_received':'Payment_Received','date_processed':'Payment_Processed','date_sent_to_accounting':'Payment_toAccounting',
                     'material_allowance':'Material_Allowance','balance':'Payment_Balance','estimate_link':'Comments','Vendor Name':'Contractor_Name','Material Certificate:':'Material_Certificate',
                     'Punch List to Contractor:':'Project_PunchList_Sent','Punch List Complete:':'Project_PunchList_Complete','Closing Conference:':'Project_Closing_Conference','Time Charges Stopped:':'Project_Charges_Stopped',
                     'Contractor Sent Final Quantities:':'Project_FinalQuantities_Sent','Contractor Accepts Final Quantities:':'Project_FinalQuantities_Accept','Date Accepted':'Project_Date_Accepted'}, inplace=True)

# Drop duplicate and irrelevant columns
df.drop(['pi_numbers','last_voucher_processed','ContractID:','Description:','Project#(s):','PI Numbers(s):','Source:'], axis=1, inplace=True)

def parse_money(value): 
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value)
df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
df['Payment_Balance'] = df['Payment_Balance'].apply(parse_money)
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)
df['Material_Allowance'] = df['Material_Allowance'].apply(parse_money)

# Convert all date columns to a standard format MM-DD-YYYY
df['Project_Comp_Est'] = pd.to_datetime(df['Project_Comp_Est'],format='%b-%d-%Y',errors='coerce')
df['Project_Comp_Substantial'] = pd.to_datetime(df['Project_Comp_Substantial'],format='%b-%d-%Y',errors='coerce')
df['Payment_Date'] = pd.to_datetime(df['Payment_Date'],format='%b-%d-%Y',errors='coerce')
df['Payment_Approved'] = pd.to_datetime(df['Payment_Approved'],format='%b-%d-%Y',errors='coerce')
df['Payment_Received'] = pd.to_datetime(df['Payment_Received'],format='%b-%d-%Y',errors='coerce')
df['Payment_Processed'] = pd.to_datetime(df['Payment_Processed'],format='%b-%d-%Y',errors='coerce')
df['Payment_toAccounting'] = pd.to_datetime(df['Payment_toAccounting'],format='%b-%d-%Y',errors='coerce')
df['Project_Letting'] = pd.to_datetime(df['Project_Letting'],format='%b-%d-%Y',errors='coerce')
df['Project_PunchList_Sent'] = pd.to_datetime(df['Project_PunchList_Sent'],format='%b-%d-%Y',errors='coerce')
df['Project_PunchList_Complete'] = pd.to_datetime(df['Project_PunchList_Complete'],format='%b-%d-%Y',errors='coerce')
df['Project_Closing_Conference'] = pd.to_datetime(df['Project_Closing_Conference'],format='%b-%d-%Y',errors='coerce')
df['Project_Charges_Stopped'] = pd.to_datetime(df['Project_Charges_Stopped'],format='%b-%d-%Y',errors='coerce')
df['Project_FinalQuantities_Sent'] = pd.to_datetime(df['Project_FinalQuantities_Sent'],format='%b-%d-%Y',errors='coerce')
df['Project_FinalQuantities_Accept'] = pd.to_datetime(df['Project_FinalQuantities_Accept'],format='%b-%d-%Y',errors='coerce')
df['Project_Date_Accepted'] = pd.to_datetime(df['Project_Date_Accepted'],format='%b-%d-%Y',errors='coerce')
df['Material_Certificate'] = pd.to_datetime(df['Material_Certificate'],format='%b-%d-%Y',errors='coerce')


df['Project_Comp_Est'] = df['Project_Comp_Est'].dt.strftime('%m-%d-%Y')
df['Project_Comp_Substantial'] = df['Project_Comp_Substantial'].dt.strftime('%m-%d-%Y')
df['Payment_Date'] = df['Payment_Date'].dt.strftime('%m-%d-%Y')
df['Payment_Approved'] = df['Payment_Approved'].dt.strftime('%m-%d-%Y')
df['Payment_Received'] = df['Payment_Received'].dt.strftime('%m-%d-%Y')
df['Payment_Processed'] = df['Payment_Processed'].dt.strftime('%m-%d-%Y')
df['Payment_toAccounting'] = df['Payment_toAccounting'].dt.strftime('%m-%d-%Y')
df['Project_Letting'] = df['Project_Letting'].dt.strftime('%m-%d-%Y')
df['Project_PunchList_Sent'] = df['Project_PunchList_Sent'].dt.strftime('%m-%d-%Y')
df['Project_PunchList_Complete'] = df['Project_PunchList_Complete'].dt.strftime('%m-%d-%Y')
df['Project_Closing_Conference'] = df['Project_Closing_Conference'].dt.strftime('%m-%d-%Y')
df['Project_Charges_Stopped'] = df['Project_Charges_Stopped'].dt.strftime('%m-%d-%Y')
df['Project_FinalQuantities_Sent'] = df['Project_FinalQuantities_Sent'].dt.strftime('%m-%d-%Y')
df['Project_FinalQuantities_Accept'] = df['Project_FinalQuantities_Accept'].dt.strftime('%m-%d-%Y')
df['Project_Date_Accepted'] = df['Project_Date_Accepted'].dt.strftime('%m-%d-%Y')
df['Material_Certificate'] = df['Material_Certificate'].dt.strftime('%m-%d-%Y')

# convert "Payment_Number" column to integer type to avoid duplicate records when integrated with duckdb
df['Payment_Number'] = df['Payment_Number'].astype(int)

EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Georgia\data_store_GA.duckdb"
table_name = "GA_DOT"

# Current scraped data
scraped_data = df

# Connect to DuckDB
con = duckdb.connect(db_file)

# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Contractor_Name  TEXT,
Contract_Number  TEXT,
Project_Number  TEXT,
Contract_Source	 TEXT,
Payment_Number  INTEGER,
Project_Description	TEXT,
Project_Comp_Est  TEXT,
Project_Comp_Substantial  TEXT,	

Payment_Amount DOUBLE,	
Project_Cost_Total  DOUBLE,	
Payment_Amount_Total  DOUBLE,
Payment_Total_Percent  TEXT,
Payment_Amount_Percent  FLOAT,
Payment_Balance  DOUBLE,
Material_Allowance DOUBLE,

Pull_Date_Initial TEXT,
Payment_Date  TEXT,
Payment_Approved  TEXT,
Payment_Received  TEXT,
Payment_Processed  TEXT,
Payment_toAccounting  TEXT,
Comments  TEXT,

Project_Letting TEXT,
Material_Certificate  TEXT,	
Project_PunchList_Sent  TEXT,
Project_PunchList_Complete	TEXT,
Project_Closing_Conference  TEXT,
Project_Charges_Stopped  TEXT,
Project_FinalQuantities_Sent  TEXT,
Project_FinalQuantities_Accept TEXT,	
Project_Date_Accepted TEXT
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()

# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial','Payment_Amount_Percent'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Project_Number", "Pull_Date_Initial"], ascending=[True,False])
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conersion errors
    combined_data = combined_data[correct_order]

else:
    combined_data = scraped_data
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Contract_Number", "Pull_Date_Initial"], ascending=[True,False])
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conersion errors
    combined_data = combined_data[correct_order]

# Replace the table with the updated data
print(combined_data)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Close connection
con.close()
# send_email("Python Script Execution Successful", "The job was executed successfully.")
print("Georgia scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'Georgia scraping completed and DUCKDB file updated Successfully.')

In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("Georgia", "data_store_ga", "GA_DOT")

In [ ]:
# read data from corresponding duckdb table. Specify State, file and table name respectively
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Georgia\Monthly\ga_bulk_pipeline_aprilS.xlsx")

Oklahoma Pipeline 

In [ ]:
# Automatic driver installer
service = Service(ChromeDriverManager().install())
# URL of the site
url = 'https://www.odot.org/CONTRACTADMIN/ESTIMATES/'

# Email configuration
SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
SMTP_PORT = 587  # Usually 587 for TLS
EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECIPIENT
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email notification sent successfully.")
        logging.info("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")
        logging.error(f"Failed to send email notification: {e}")

# contract_numbers_list = test_list_ok[773:1070] 
bulk_contracts = True
# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
original_window = driver.window_handles[0]
row_data_list_ok = []
header_data_ok = ["contract_id", "payment_number","date_let","ntp_eff_date","pay_period","date_awarded","date_work_began","org_cont_time","date_cont_executed","date_time_stopped","current_time_charged","date_ntp_issued",
                  "completion_date","current_time_allowed","general_liability_exp","workman_comp_exp","percent_time_used","spec_year","date_approved","bid_amount","funds_available_bid_co","percent_complete",
                  "unearned_balance", "total_to_date","prev_to_date","this_estimate", "project_numbers", "primary_job_num", "cont_desp","primary_county","road_name","prime_cont","surety_company"]

try:
    if bulk_contracts:
        all_contracts = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main"]/div/a'))
        )
        all_contracts_list = []
        for row in all_contracts:
            all_contracts_list.append(row.text)
        contract_numbers = all_contracts_list
    elif not bulk_contracts:
        # contract_numbers = contract_numbers_list
        contract_numbers = contract_numbers

    for i,value in enumerate(contract_numbers[614:624]):
        
        # Toggle the radio button filter
        contract_id_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "filt1"))
        )
        contract_id_button.click()

        # Find the contract number input box and enter the contract number
        contract_id_input = driver.find_element(By.ID, "filt1v")
        contract_id_input.clear()
        contract_id_input.send_keys(value)
        apply = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="menuBar"]/input[8]'))
        )
        apply.click()
        # Submit the search form
        try:
            contract_id_link = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.LINK_TEXT, value))
            )
            contract_id_link.click()            
            all_reports = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="main"]/table/tbody/tr/td'))
            )
            report_counter = 0
            for i in range(len(all_reports) - 1, -1, -1): #iterate backwards through the list
                if report_counter > 3: # fetch last 4 reports only since historical data has been scraped already
                    break
                row = all_reports[i].find_element(By.XPATH,"./a")
                if "index" not in row.get_attribute('textContent') and row.get_attribute('textContent').isdigit(): # if only the latest report is needed, the loop should be ended the first time the condition is satisfied. conditional logic can be improved
                    est_num = row.get_attribute('textContent')
                    report = row
                    report.click()
                    time.sleep(1) # addresses race conditions. Expected issue to be resolved- WebDriverException: no such execution context 
                    # change the window of the driver since new tab is opened
                    new_window_handle = [handle for handle in driver.window_handles if handle != original_window][-1]
                    driver.switch_to.window(new_window_handle)
                    table = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, '/html/body/table[3]/tbody/tr'))
                    )
            
                    current_data = []
                    current_data.append(value)
                    current_data.append(est_num)
                    for row in table:
                        cells = row.find_elements(By.XPATH, './td')
                        counter = 0
                        for cell in cells:
                            if counter % 2 != 0: # If the iteration is odd
                                current_data.append(cell.text)
                            counter += 1
                    if len(current_data) < 18:
                        driver.close()
                        driver.switch_to.window(original_window)
                        continue
                    del current_data[18] # Removes extra empty values from empty cells
                    
                    bid_amount = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[1]/td[2]').text
                    funds_available = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[2]/td[2]').text
                    percent_complete = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[3]/td[2]').text
                    unearned_balance = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[4]/td[2]').text
                    total_to_date = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[9]/td[4]').text
                    prev_to_date = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[9]/td[5]').text
                    this_estimate = driver.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[9]/td[6]').text

                    current_data.append(bid_amount)
                    current_data.append(funds_available)
                    current_data.append(percent_complete)
                    current_data.append(unearned_balance)
                    current_data.append(total_to_date)
                    current_data.append(prev_to_date)
                    current_data.append(this_estimate)
                    

                    additional_info_table = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, '/html/body/table[2]/tbody/tr/td[2]'))
                    )
                    for j, row in enumerate(additional_info_table):
                        current_data.append(row.text)
                    del current_data[-3:-1]
                    if len(current_data) != 33:
                        print("check data for contract id", value)
                    else:
                        row_data_list_ok.extend([current_data])
                    driver.close()
                    driver.switch_to.window(original_window)
                    report_counter += 1
            est_portal_link = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.LINK_TEXT, 'Estimates Portal'))
                )
            est_portal_link.click()
            
        except TimeoutException:    
            print(value,"Not Found")
        except InvalidArgumentException:   # When true it indicates that the input contract ID is a numeric data type and needs to be changed to string
            print(value,"Not Found - change to string dtype")

finally:
    # Close the browser
    driver.quit()
# Create a dataframe   
OK_DOT_Data = pd.DataFrame(data=row_data_list_ok, columns = header_data_ok)
# POST PROCESSING
df = OK_DOT_Data.copy()
df.rename(columns = {'contract_id':'Contract_Number','payment_number':'Payment_Number','project_numbers':'Project_Number','date_let':'Project_Letting',
                     'date_awarded':'Project_Bid_Awarded','ntp_eff_date':'Project_Start_Effective','date_work_began':'Project_Start_Actual',
                     'cont_desp':'Project_Description','pay_period':'Payment_Work_Period','bid_amount':'Project_Cost_Total','funds_available_bid_co':'Project_Cost_Adjusted',
                     'completion_date':'Project_Comp_Final','date_time_stopped':'Project_Charges_Stopped','surety_company':'Bond_Company_Name',
                     'prime_cont':'Contractor_Name','this_estimate':'Payment_Amount','date_approved':'Payment_Approved','total_to_date':'Payment_Amount_Total',
                     'unearned_balance':'Payment_Balance','percent_complete':'Payment_Total_Percent'}, inplace=True)


def parse_money(value):
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value)

df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
df['Project_Cost_Adjusted'] = df['Project_Cost_Adjusted'].apply(parse_money)
df['Payment_Balance'] = df['Payment_Balance'].apply(parse_money)
df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
df['prev_to_date'] = df['prev_to_date'].apply(parse_money)
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)

# convert "Payment_Number" column to integer type to avoid duplicate records when integrated with duckdb
df['Payment_Number'] = df['Payment_Number'].astype(int)

df['org_cont_time'] = pd.to_numeric(df['org_cont_time'], errors='coerce')
df['current_time_allowed'] = pd.to_numeric(df['current_time_allowed'], errors='coerce')

# replace 0 with blanks for the following date columns
df[['general_liability_exp', 'workman_comp_exp']] = df[['general_liability_exp', 'workman_comp_exp']].replace(0, '')
EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Oklahoma\data_store_OK.duckdb"
table_name = "OK_DOT"

# Current scraped data
scraped_data = df.copy()

# Connect to DuckDB
con = duckdb.connect(db_file)
# Fields to be calculated


# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Contract_Number  TEXT,
Project_Number  TEXT,
primary_job_num TEXT,
Payment_Number  INTEGER,
Project_Description	TEXT,
primary_county TEXT,
Contractor_Name TEXT,
Bond_Company_Name TEXT,
road_name TEXT,

org_cont_time DOUBLE,
date_cont_executed TEXT,
Project_Charges_Stopped TEXT,
current_time_charged TEXT,
current_time_allowed DOUBLE,
general_liability_exp TEXT,
workman_comp_exp TEXT,
percent_time_used TEXT,
spec_year TEXT,

Project_Cost_Total DOUBLE,
Project_Cost_Adjusted DOUBLE,
Project_Cost_Additions DOUBLE,

Payment_Balance DOUBLE,
Payment_Amount_Total DOUBLE,
prev_to_date DOUBLE,
Payment_Amount DOUBLE,
Payment_Total_Percent TEXT,
Payment_Amount_Percent DOUBLE,


Pull_Date_Initial TEXT,
Payment_Approved TEXT,
Payment_Date TEXT,
Project_Letting TEXT,
Project_Bid_Awarded TEXT,
Project_Start_Effective TEXT,
Project_Start_Actual TEXT,
Payment_Work_Period TEXT,
Project_Comp_Final TEXT,
date_ntp_issued TEXT,
Project_Comp_Est TEXT,
Project_Comp_Substantial TEXT
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()

# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial','Project_Cost_Additions','Payment_Amount_Percent'
                                                                                     'Project_Comp_Est','Project_Comp_Substantial','Payment_Date'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Number"], ascending=[True,False])

    # Calculate derived fields safely
    combined_data['Project_Cost_Additions'] = combined_data['Project_Cost_Adjusted'] - combined_data['Project_Cost_Total']
    combined_data['Payment_Amount_Percent'] = round(
    combined_data['Payment_Amount'] / combined_data['Payment_Amount_Total'].replace(0, pd.NA) * 100, 2
    )
    # Extract the last date string using string slicing (equivalent to SUBSTR)
    combined_data['Payment_Date'] = combined_data['Payment_Work_Period'].str[-10:]

    # Convert and calculate date-based fields
    combined_data['date_ntp_issued'] = pd.to_datetime(combined_data['date_ntp_issued'], errors='coerce')
    combined_data['Project_Comp_Est'] = combined_data['date_ntp_issued'] + pd.to_timedelta(combined_data['org_cont_time'], unit='D')
    combined_data['Project_Comp_Substantial'] = combined_data['date_ntp_issued'] + pd.to_timedelta(combined_data['current_time_allowed'], unit='D')

    # Format dates back to strings
    combined_data['Project_Comp_Est'] = combined_data['Project_Comp_Est'].dt.strftime('%m/%d/%Y')
    combined_data['Project_Comp_Substantial'] = combined_data['Project_Comp_Substantial'].dt.strftime('%m/%d/%Y')
    combined_data['date_ntp_issued'] = combined_data['date_ntp_issued'].dt.strftime('%m/%d/%Y')

    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

else:
    combined_data = scraped_data.copy()
    # Post processing before loading into duckdb
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Number"], ascending=[True,False])
    # Calculate derived fields safely
    combined_data['Project_Cost_Additions'] = combined_data['Project_Cost_Adjusted'] - combined_data['Project_Cost_Total']
    combined_data['Payment_Amount_Percent'] = round(
    combined_data['Payment_Amount'] / combined_data['Payment_Amount_Total'].replace(0, pd.NA) * 100, 2
    )
    # Extract the last date string using string slicing (equivalent to SUBSTR)
    combined_data['Payment_Date'] = combined_data['Payment_Work_Period'].str[-10:]

    # Convert and calculate date-based fields
    combined_data['date_ntp_issued'] = pd.to_datetime(combined_data['date_ntp_issued'], errors='coerce')
    combined_data['Project_Comp_Est'] = combined_data['date_ntp_issued'] + pd.to_timedelta(combined_data['org_cont_time'], unit='D')
    combined_data['Project_Comp_Substantial'] = combined_data['date_ntp_issued'] + pd.to_timedelta(combined_data['current_time_allowed'], unit='D')

    # Format dates back to strings
    combined_data['Project_Comp_Est'] = combined_data['Project_Comp_Est'].dt.strftime('%m/%d/%Y')
    combined_data['Project_Comp_Substantial'] = combined_data['Project_Comp_Substantial'].dt.strftime('%m/%d/%Y')
    combined_data['date_ntp_issued'] = combined_data['date_ntp_issued'].dt.strftime('%m/%d/%Y')

    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

# Replace the table with the updated data
print(combined_data.shape)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Close connection
con.close()
send_email("Python Script Execution Successful", "The job was executed successfully.")
print("Oklahoma scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'Oklahoma scraping completed and DUCKDB file updated Successfully.')


In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("Oklahoma", "data_store_OK", "OK_DOT")                                                       

In [ ]:
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Oklahoma\Monthly\OK_bulk_pipeline_april.xlsx")

Washington Pipeline

In [ ]:
# Email configuration
SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
SMTP_PORT = 587  # Usually 587 for TLS
EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECIPIENT
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email notification sent successfully.")
        logging.info("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")
        logging.error(f"Failed to send email notification: {e}")
# Type the desired file name and save
file_path = r"C:\Users\TarunPongulaty\Downloads\wa_active_contracts_list.pdf" 
# Automatic driver installer
service = Service(ChromeDriverManager().install())
# Set up the WebDriver
options = webdriver.ChromeOptions() 
options.add_argument("--start-maximized") # Ensures the browser is fullscreen
# URL of the site
url = 'https://www.wsdot.wa.gov/publications/fulltext/construction/projectreports/Active.pdf-en-us.pdf'
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
# original_window = driver.window_handles[0]
try:
        time.sleep(3)
        # Simulate pressing Ctrl+S to open "Save As" dialog
        pyautogui.hotkey('ctrl', 's')
        time.sleep(2)
         # Update with your desired location and name
        pyautogui.typewrite(file_path)
        time.sleep(1)  # Small delay for typing

        # Press 'Enter' to save
        pyautogui.press('enter')

        time.sleep(2)
        pyautogui.press('left')  # Move focus to the "Yes" button
        time.sleep(0.5)  # Small delay for smooth navigation
        pyautogui.press('enter')

        # Wait for the file to be saved
        time.sleep(5)
finally:
    # Close the browser
    driver.quit()

#working pdf reader

tables = tabula.read_pdf(file_path, pages='all', multiple_tables=True)
df = pd.concat(tables, ignore_index=True)
df['Contract\rNumber'] = df['Contract\rNumber'].astype(str).apply(lambda x: x.split('.')[0].zfill(6)) # makes sure leading zeros are retained by fixing the length of ID to 6. In the website it is mentioned that the contract numbers are 6 in length
contract_numbers_wa = df['Contract\rNumber'].to_list() # 'split('.')[0]' is used above as the pdf reader adds decimals to numbers in some cases



In [ ]:
# Automatic driver installer
service = Service(ChromeDriverManager().install())

# URL of the site
url = 'https://remoteapps.wsdot.wa.gov/construction/project/progress/'

# Contract number to search for
contract_numbers = contract_numbers_wa 
# contract_numbers = ['009446']           
header_data_wa = []
# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()
row_data_list_wa = []
try:
    for i,value in enumerate(contract_numbers):
        # Wait for the contract number input box to be present
    
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "txtContractNumber"))
        )

        # Find the contract number input box and enter the contract number
        contract_number_input = driver.find_element(By.ID, "txtContractNumber")
        contract_number_input.clear()
        contract_number_input.send_keys(value)
        

        # Submit the search form
        contract_number_input.send_keys(Keys.RETURN)
        # Locate the dropdown element for payment dates
        try:
            payment_date_options = WebDriverWait(driver, 100).until(
                        EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="select-payment-date"]/option'))
                    )
            report_counter = 0
            # iterate over the payment date options:
            for option in payment_date_options:
                if report_counter > 4: # fetch last 4 reports only since historical data has been scraped already
                    break
                option.click()
                try:
                    WebDriverWait(driver, 300).until(
                        EC.presence_of_element_located((By.ID, 'Page1'))
                    )
                    table1 = WebDriverWait(driver, 300).until(
                        EC.visibility_of_element_located((By.XPATH, '//table[@class="S45"]'))
                    )
                    # Find all headers from 2 tables and append them
                    if not header_data_wa:
                        # Find all cells in the row
                        for row in table1.find_elements(By.XPATH,'.//tr'):    
                            cells = row.find_elements(By.XPATH,'.//td')    
                            # Extract headers only from table 1    
                            header_data_wa.extend([cells[0].get_attribute("textContent").strip()]) 
                        del header_data_wa[2]               
                        # Extract header only from table 2    
                        header = driver.find_element(By.XPATH,'//table[@id="List1"]/tbody/tr[last()]/td')
                        header_data_wa.extend([header.get_attribute("textContent").strip()])

                    # Extract row data from table 1 and 2
                    row_data_list = []
                    for row in table1.find_elements(By.XPATH,'.//tr'):    
                        cells = row.find_elements(By.XPATH,'.//td')   
                        # Extract records only from table 1 
                        if len(cells) >=2:          
                            values = cells[1].get_attribute("textContent")
                            if not values:
                                values = "N/A"  # Use a default value
                        if any(values):
                            row_data_list.append(values)
                        
                    del row_data_list[2] # Deletes the empty element extracted  due to extra tag containing just space in html structure
                    # Extract total from table 2
                    row_data = driver.find_element(By.XPATH,'//table[@id="List1"]/tbody/tr[last()]/td[last()]')
                    row_data_list.append(row_data.get_attribute("textContent"))
                    row_data_list_wa.extend([row_data_list])
                    report_counter += 1
        
                except NoSuchElementException:
                    print("No Data for", value)    
                except TimeoutException:
                    print(value,"some payment estimate data Not Found")
                except Exception as e:
                    print(f"Error processing payment estimate: {e}")
        except TimeoutException:
                    print("Not Found for", value)
        
        # delay_seconds = random.uniform(2,10)
        # time.sleep(delay_seconds)
        driver.get(url)
except TimeoutException:
    print("Website not responding. Re-run")
finally:
    # Close the browser
    driver.quit()
    
WA_DOT_data = pd.DataFrame(data=row_data_list_wa, columns = header_data_wa)


In [ ]:
# POST PROCESSING
df = WA_DOT_data.copy()

# Split "Contract Number:" into 'Contract_Number' and 'Project_Description'
split_columns = df['Contract Number:'].str.split(' ', n=1, expand=True)
split_columns.columns = ['Contract_Number', 'Project_Description']
df = pd.concat([df, split_columns], axis = 1)
del df['Contract Number:']


df.rename(columns = {'Prime Contractor:':'Contractor_Name','Project Engineer:':'Engineer_Name','Estimate Number(s):': 'Estimate_Number_Original',
                     'Payment Date:':'Payment_Date','Overall - Total':'Payment_Amount','Last Date of Work This Payment:':'Last_Date_of_Work_This_Payment'}, inplace=True)

# Create payment work period column from prior last date of work this payment to payment date
df["Payment_Work_Period"] = df["Last_Date_of_Work_This_Payment"].str.cat(df["Payment_Date"], sep=" to ", na_rep="Unknown")

def parse_money(value):
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value)

# format money columns to numeric for calculations ahead
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)

# Create scraping pull date column
EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Washington\data_store_WA.duckdb"
table_name = "WA_DOT"

# Current scraped data
scraped_data = df

# Connect to DuckDB
con = duckdb.connect(db_file)

# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Contract_Number  TEXT,
Contractor_Name  TEXT,
Engineer_Name  TEXT,
Payment_Number  INT,
Estimate_Number_Original  TEXT,
Project_Description	TEXT,
Payment_Date  TEXT,
Last_Date_of_Work_This_Payment  TEXT,
Payment_Work_Period  TEXT,
Payment_Amount DOUBLE,	
Payment_Amount_Total  DOUBLE,
Project_Cost_Total	DOUBLE,
Payment_Balance DOUBLE,
Pull_Date_Initial TEXT,
Payment_Amount_Percent FLOAT,
Payment_Total_Percent  FLOAT
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()

# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    combined_data['Estimate_Number_Original'] = combined_data['Estimate_Number_Original'].str.strip()
    # find duplicates by all columns except the columns below. Compared to other DOT'S WA requires different dedup columns since the 'payment amount total' is calculated post scraping 
    # and if minor changes such as 'engineer name' or 'project description' occur individually that will not be captured as it would lead to erroneous payment amount total. However if these changes happen in tandem with 
    # any one of the columns identified for deduplication ('Contract_Number','Estimate_Number_Original','Payment_Amount','Payment_Date') then those changes will be captured
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Project_Cost_Total','Payment_Balance','Pull_Date_Initial','Project_Description',
                                                                                     'Payment_Amount_Percent','Payment_Number','Payment_Total_Percent','Contractor_Name',
                                                                                     'Payment_Amount_Total', 'Engineer_Name','Payment_Work_Period','Last_Date_of_Work_This_Payment'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data['Payment_Date'] = pd.to_datetime(combined_data['Payment_Date'], errors="coerce")
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    # Sort the DataFrame
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Date","Pull_Date_Initial"], ascending=[True,True,False],na_position='first')
    # Assign a payment estimate number
    combined_data["Payment_Number"] = combined_data.groupby("Contract_Number").cumcount(ascending=True) 
    # Calculate the Current Payment Amount Total
    combined_data['Payment_Amount_Total'] = combined_data[combined_data['Payment_Amount'] >= 0].groupby("Contract_Number")['Payment_Amount'].cumsum()
    # Revert the formatting of the date columns
    combined_data["Payment_Date"] = combined_data["Payment_Date"].dt.strftime('%m/%d/%Y')
    combined_data["Payment_Date"] = combined_data["Payment_Date"].fillna("N/A")
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Calculate Project_Cost_Total by populating the the most recent Payment_Amount_Total. Note that this approach doesn't yield the 'True' Project value.
    combined_data['Project_Cost_Total'] = combined_data.groupby('Contract_Number')['Payment_Amount_Total'].transform('max')
    # Calculate Payment_Balance
    combined_data['Payment_Balance'] = combined_data['Project_Cost_Total'] - combined_data['Payment_Amount_Total']
    # Calculate Payment_Total_Percent
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]    

else:
    combined_data = scraped_data.copy()
    # Post processing before loading into duckdb
    combined_data['Estimate_Number_Original'] = combined_data['Estimate_Number_Original'].str.strip()
    combined_data['Payment_Date'] = pd.to_datetime(combined_data['Payment_Date'], errors="coerce")
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Date","Pull_Date_Initial"], ascending=[True,True,False],na_position='first')
    # Assign a payment estimate number
    combined_data["Payment_Number"] = combined_data.groupby("Contract_Number").cumcount(ascending=True)
    # Calculate the Current Payment Amount Total
    combined_data['Payment_Amount_Total'] = combined_data[combined_data['Payment_Amount'] >= 0].groupby("Contract_Number")['Payment_Amount'].cumsum()
    # Revert the formatting of the date columns
    combined_data["Payment_Date"] = combined_data["Payment_Date"].dt.strftime('%m/%d/%Y')
    combined_data["Payment_Date"] = combined_data["Payment_Date"].fillna("N/A")
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Calculate Project_Cost_Total by populating the the most recent Payment_Amount_Total. Note that this approach doesn't yield the 'True' Project value.
    combined_data['Project_Cost_Total'] = combined_data.groupby('Contract_Number')['Payment_Amount_Total'].transform('max')
    # Calculate Payment_Balance
    combined_data['Payment_Balance'] = combined_data['Project_Cost_Total'] - combined_data['Payment_Amount_Total']
    # Calculate Payment_Total_Percent
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]


# Replace the table with the updated data
print(combined_data)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Close connection
con.close()
send_email("Python Script Execution Successful", "The job was executed successfully.")
print("Washington scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'Washington scraping completed and DUCKDB file updated Successfully.')

In [ ]:
data = get_data("Washington", "data_store_WA", "WA_DOT") 

In [ ]:
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Washington\Monthly\wa_bulk_pipeline_april_updated.xlsx")

Illinois Pipeline 

In [ ]:
# Configuração de Logging
logging.basicConfig(
    filename='Illinois_scraping.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)
# Email configuration
SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
SMTP_PORT = 587  # Usually 587 for TLS
EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECIPIENT
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email notification sent successfully.")
        logging.info("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")
        logging.error(f"Failed to send email notification: {e}")
        
# Retrieve contract numbers

# WebDriver Manager will handle everything automatically
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
# URL of the site
url = 'https://apps1.dot.illinois.gov/WCP/PEstimate/PEstimateSearch'

contract_numbers_il = []
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window() # resolves issue where dropdown buttons on this page are not visible when window is minimized

try:
    time.sleep(10)
    wait = WebDriverWait(driver, 20)
    contract_num_button = wait.until(
        EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div/div/form[2]/div/div[2]/span/button'))
    )
    contract_num_button.click()

    contract_num_list = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[4]/div/div/div[2]/ul/li'))
    )

    for i, row in enumerate(contract_num_list):
        number = row.get_attribute("textContent")
        contract_numbers_il.append(number)

finally:
    driver.quit()

# Get associated Contract information
driver = webdriver.Chrome(service=service)
# URL of the site
url = 'https://apps1.dot.illinois.gov/WCP/'


# Create connection to duckdb table
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Illinois\data_store_IL.duckdb"
con = duckdb.connect(db_file)
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM IL_DOT").df()
con.close()
df1 = existing_data.drop_duplicates(subset=["Contract_Number"], keep='first')

# Convert 'Payment_Date' to datetime format
df1['Payment_Date'] = pd.to_datetime(df1['Payment_Date'], format='%m/%d/%Y', errors='coerce')

# Convert 'Percent_Complete' to numeric format (remove '%' and convert to float)
df1['Payment_Total_Percent'] = df1['Payment_Total_Percent'].str.rstrip('%').astype(float)

# Define the filter conditions
condition1 = (df1['Payment_Date'].dt.year <= 2024) & (df1['Payment_Total_Percent'] >= 100)
condition2 = (df1['Payment_Date'].dt.year <= 2023) & (df1['Payment_Total_Percent'] >= 95)
condition3 = df1['Payment_Date'].dt.year <= 2021

# Apply filtering: Keep only rows that satisfy any of the conditions
filtered_df = df1[(condition1 | condition2 | condition3)]
inactive_contracts = filtered_df['Contract_Number'].to_list()
# Filter out inactive contracts for scraping
contract_numbers_il_updated = [num for num in contract_numbers_il if num not in inactive_contracts]
print(len(contract_numbers_il_updated))

# Contract number to search for
contract_numbers = contract_numbers_il_updated

# Start a new browser session
driver.get(url)
driver.maximize_window() 
row_data_list_il = []
header_data_il = ["percent_completed", "current_report","from_date", "to_date", "Contract_Awarded_Amount", "Additions",
                  "Deductions", "Total_Adjusted_Contract_Value", "Total_Amount_Due_to_Date", "Payment_Number", "total", "contract_number",
                  "state_job", "dot_vendor", "district/county", "il_project", "letting_date", "route", "airport/section", "section",
                  "project", "payee", "scope"]

try:
    wait = WebDriverWait(driver, 100)
    for i,value in enumerate(contract_numbers):
        # Wait for the contract number input box to be present
        state_no_input = WebDriverWait(driver, 15).until(     
            EC.visibility_of_element_located((By.ID, 'ContractNumber')) 
        ) 
        state_no_input.send_keys(value)
        state_no_input.send_keys(Keys.RETURN)
        try:
            try:
                contract_number_link = WebDriverWait(driver, 7).until(
                    EC.presence_of_element_located((By.LINK_TEXT, value))
                )
                time.sleep(3) # possibly resolves staleelement error
                contract_number_link.click()
            except StaleElementReferenceException:
                print(value,"StaleElementReferenceException error. Rescrape")
                continue
            contract_info = []
            contractor_info = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="EstimateChoiceContractorInfo"]'))
            )
            time.sleep(5)
            all_invoices = WebDriverWait(driver,15).until(
                EC.presence_of_all_elements_located((By.XPATH, "/html/body/div[1]/div[2]/div[1]/div[3]/div/div/div[1]/ul/li/span[@class='k-link']"))
            )
            
            data = contractor_info.find_elements(By.XPATH, './/div[label]')

            if len(data) != 12:
                print("possible data column mismatch at following contract and iteration", value,i)

            for info in data:
                cell = info.find_element(By.XPATH, "./div")
                contract_info.append(cell.get_attribute("textContent").strip())
    
            # iterate through all transactions
            report_counter = 0
            try:
                for j, invoice in enumerate(all_invoices):
                    if report_counter > 2: # fetch latest 3 reports only since historical data has been scraped already
                        break
                    print("current value, iteration, and invoice", value, i, j)
                    if j == 0: # This condition resolves ElementClickInterceptedException error which can occur during first iteration
                        WebDriverWait(driver, 25).until(
                        EC.visibility_of_element_located((By.XPATH, './/div[@ID="TabStrip"]'))
                        )
                        current_id = "TabStrip-" + str(j+1)
                        container = wait.until(
                            EC.visibility_of_element_located((By.ID, current_id)) # modify this to remove sleep(add class container)
                        )
                    invoice.click()
                    time.sleep(1)   
                    # Wait for content to load
                    if j > 0:
                        wait.until(
                            EC.visibility_of_element_located((By.XPATH, './/div[@ID="TabStrip"]')) # modify this to remove sleep(add class container)
                        )
                        current_id = "TabStrip-" + str(j+1)
                        container = wait.until(
                            EC.visibility_of_element_located((By.ID, current_id)) # modify this to remove sleep(add class container)
                        )
                    time.sleep(1)

                    # Retrieve dates and percent complete information
                    current_data = []
                    payment_data1 = container.find_element(By.XPATH, './/div[@class="container mt-3"]')
                    for cell in payment_data1.find_elements(By.XPATH,'.//div[@class="form-group"]/div'):
                        current_data.append(cell.get_attribute("textContent").strip())

                    # Retrieve additional contract payment info such as 'contract awarded amount', 'additions', 'Total Adjusted Contract Value' and more
                    payment_data2 = container.find_element(By.XPATH, './div[3]')
                    payment_data2 = payment_data2.find_element(By.XPATH, './/table[1]/tbody/tr')
                    for cell in payment_data2.find_elements(By.XPATH, './td'):
                        current_data.append(cell.get_attribute("textContent").strip())

                    # Retrieve Total Cost and payment number of corresponding invoice
                    payment_number = container.find_element(By.XPATH, "./div[3]//div[@class='form-group'][last()]/div[@class='form-group row']/div[2]")
                    Total = container.find_element(By.XPATH, "./div[3]//div[@class='form-group'][last()]/div[@class='form-group row']/div[last()]")
                    current_data.append(payment_number.text)
                    current_data.append(Total.text)
                    combined_data = current_data + contract_info
                    row_data_list_il.extend([combined_data])      
                    report_counter += 1

            except ElementClickInterceptedException:
                print("ElementClickInterceptedException error, rescrape", value)
            except TimeoutException:        
                print(value,"Invoice loading timed out")
            except StaleElementReferenceException:
                print(value,"Loading Error")

        except TimeoutException:        
            print(value,"Not Found")
                
        driver.get(url)

# Close the browser
finally:
    driver.quit()
  

In [ ]:
#DuckDB Integration
il_dot_data = pd.DataFrame(data=row_data_list_il, columns = header_data_il)
del il_dot_data['section']
del il_dot_data['il_project']

# POST PROCESSING
df = il_dot_data.copy()
df = df[df['Payment_Number'].str.isnumeric()]
df['Payment_Number'] = df['Payment_Number'].astype(int)
# Split "district/county" into 'Project_Location_District' and 'Project_Location_County'
df[['Project_Location_District', 'Project_Location_County']] = df['district/county'].str.extract(r'([\w\s]+?)\s*-\s*\d{3}\s*\(\s*([\w\s.]+?)\s*\)')
del df['district/county']

# Create payment work period column from "from_date" and "to_date"
df["Payment_Work_Period"] = df["from_date"].str.cat(df["to_date"], sep=" to ", na_rep="Unknown")
del df['from_date']
del df['to_date']

df.rename(columns = {'project':'Project_Number','scope':'Project_Description','route':'Project_Location_Route','airport/section':'Project_Location_Section',
                     'state_job':'Project_Location_StateJob','letting_date':'Project_Letting','contract_number':'Contract_Number','dot_vendor':'Contractor_DOT',
                     'total':'Payment_Amount','current_report':'Payment_Date','payee':'Contractor_Name','percent_completed':'Payment_Total_Percent',
                     'Total_Amount_Due_to_Date':'Payment_Amount_Total','Contract_Awarded_Amount':'Project_Cost_Total','Additions':'Project_Cost_Additions',
                     'Deductions':'Project_Cost_Deductions','Total_Adjusted_Contract_Value':'Project_Cost_Total_Adjusted'}, inplace=True)
def parse_money(value):
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value) if value.strip() else None 

# Convert money columns into calculable columns
df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)
df['Project_Cost_Additions'] = df['Project_Cost_Additions'].apply(parse_money)
df['Project_Cost_Deductions'] = df['Project_Cost_Deductions'].apply(parse_money)
df['Project_Cost_Total_Adjusted'] = df['Project_Cost_Total_Adjusted'].apply(parse_money)

df['Payment_Balance'] = (df['Project_Cost_Total_Adjusted'] - df['Payment_Amount_Total']).round(2)

EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Illinois\data_store_IL.duckdb"
table_name = "IL_DOT"

# Current scraped data
scraped_data = df.copy()

# Connect to DuckDB
con = duckdb.connect(db_file)

# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Contract_Number  TEXT,
Contractor_Name  TEXT,
Contractor_DOT  TEXT,
Project_Number  TEXT,
Payment_Number  INTEGER,
Project_Description	TEXT,
Project_Location_Route TEXT,
Project_Location_Section  TEXT,
Project_Location_StateJob  TEXT,
Project_Location_District  TEXT, 
Project_Location_County  TEXT,
Project_Letting  TEXT,
Payment_Date  TEXT,
Payment_Work_Period  TEXT,

Payment_Amount DOUBLE,	
Project_Cost_Total  DOUBLE,	
Project_Cost_Additions  DOUBLE,
Project_Cost_Deductions  DOUBLE,
Project_Cost_Total_Adjusted  DOUBLE,
Payment_Amount_Total  DOUBLE,
Payment_Total_Percent  TEXT,
Payment_Balance  DOUBLE,
Pull_Date_Initial TEXT,
Payment_Amount_Percent FLOAT,
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()

# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    # find duplicates by all columns except the columns below since they are calculated later and those columns that need not be tracked for changes if occur separately.
    # eg: for a given contract and payment number, if only "Contractor_DOT" is changed while other fields are the same we don't need to capture that unless required by Census.
    # Later, a logic can be developed to find revised payments if any
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial', 'Payment_Amount_Percent','Contractor_DOT','Project_Location_District','Project_Location_County','Payment_Balance'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Number","Pull_Date_Initial"], ascending=[True,False,False])
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

else:
    combined_data = scraped_data.copy()
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Contract_Number", "Payment_Number","Pull_Date_Initial"], ascending=[True,False,False])
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

# Replace the table with the updated data
print(combined_data)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Close connection
con.close()
send_email("Python Script Execution Successful", "The job was executed successfully.")
print("Illinois scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'Illinois scraping completed and DUCKDB file updated Successfully.')

In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("Illinois", "data_store_IL", "IL_DOT")
data.head()     

In [ ]:
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Illinois\Monthly\il_bulk_pipeline_april_2025.xlsx")

Delaware Pipeline 

In [ ]:
# Retrieve all contract numbers

service = Service(ChromeDriverManager().install())
# URL of the site
url = 'https://deldot.gov/Business/ProjectStatusQuery/'
# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
contract_numbers_del = []
header_contract_del = ["contract_number","awarded_date"]

driver.maximize_window()
time.sleep(12) # Next button is disabled initially while page is loading. Explicit wait is required
try:
    next_page_button = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="property_next"]'))
            )        
    while "disabled" not in next_page_button.get_attribute("class"): 
    # wait to load
        body = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="property"]'))
                )
        WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="property"]/tbody/tr[1]/td[2]/a'))
                )
        contract_numbers = driver.find_elements(By.XPATH, '//*[@id="property"]/tbody/tr')
        for row in contract_numbers:
            current_contract = []
            cell_2 = row.find_element(By.XPATH, "./td[2]/a")
            cell_5 = row.find_element(By.XPATH, "./td[5]")
            current_contract.append(cell_2.text)
            current_contract.append(cell_5.text)
            # if len(current_contract) < 2:
            #     current_contract.append(" ")
            contract_numbers_del.append(current_contract)
        next_page_button.click()
        next_page_button = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, '//*[@id="property_next"]'))
        ) 

finally:
    # Close the browser
    driver.quit()
del_contracts = pd.DataFrame(data=contract_numbers_del, columns = header_contract_del)
del_contracts.head(12)

In [ ]:
# Filter out inactive contracts
df_updated = del_contracts[del_contracts['awarded_date'] != '']
contract_numbers_del_filtered = df_updated['contract_number'].to_list()

# Filter out completed contracts
# Create connection to duckdb table
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Delaware\data_store_dl.duckdb"
con = duckdb.connect(db_file)
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM DL_DOT").df()
con.close()
df1 = existing_data.drop_duplicates(subset=["Project_Number"], keep='last')
print(df1.shape)
print(len(df1['Project_Number'].unique()))

# Convert 'Payment_Date' to datetime format
df1['Payment_Date'] = pd.to_datetime(df1['Payment_Date'], format='%m/%d/%Y', errors='coerce')
df1['Project_Bid_Awarded'] = pd.to_datetime(df1['Project_Bid_Awarded'], format='%m/%d/%Y', errors='coerce')

# Define the filter conditions
condition1 = (df1['Payment_Date'].dt.year <= 2024) & (df1['Payment_Total_Percent'] >= 100)
condition2 = (df1['Payment_Date'].dt.year <= 2023) & (df1['Payment_Total_Percent'] >= 95)
condition3 = df1['Payment_Date'].dt.year <= 2021
condition4 = df1['Payment_Status'] == "Final"
condition5 = (df1['Project_Bid_Awarded'].dt.year <= 2010)

# Apply filtering: Keep only rows that satisfy any of the conditions
filtered_df = df1[(condition1 | condition2 | condition3 | condition4 | condition5)]
inactive_contracts_dl = filtered_df['Project_Number'].to_list()
# Filter out inactive contracts for scraping
contract_numbers_dl_updated = [num for num in contract_numbers_del_filtered if num not in inactive_contracts_dl]
print(len(contract_numbers_dl_updated))

# Contract number to search for
contract_numbers = contract_numbers_dl_updated

In [ ]:
service = Service(ChromeDriverManager().install())

# URL of the site
url = 'https://deldot.gov/Business/ProjectStatusQuery/'
                    
contract_numbers = contract_numbers_dl_updated
# contract_numbers = ["T200507103"]
# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()
row_data_list_del = []
header_data_del = []
original_window = driver.window_handles[0]

def extract_text(cell):
                    """Extracts text from a cell, replacing <br> with new lines."""
                    return cell.get_attribute('innerHTML').replace('<br>', '\n').strip()
try:
    for t,value in enumerate(contract_numbers):
        print(f"iteration {t}th, working on {value}")
        # Wait for the contract number input box to be present
        state_no_input = WebDriverWait(driver, 15).until(     
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#stateNoFilter input')) 
        ) 
        state_no_input.send_keys(value)
        try:
            contract_number_link = WebDriverWait(driver, 70).until(
                EC.presence_of_element_located((By.LINK_TEXT, value))
            )
        
            contract_number_link.click()
        
            # Find tables
            info_table2 = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="project-info"]/div/table[2]/tbody/tr'))
            )
            info_table1 = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="project-info"]/div/table[1]/tbody/tr'))
            )
            
            try:
            # Extract the table headers
                if t == 0 or not header_data_del:
                    payment_header = driver.find_element(By.XPATH, '//table[@id="paymentInfo"]/thead/tr').find_elements(By.XPATH,'.//th') 
                    for h,header in enumerate(payment_header):
                        if h != 1 and h != 2:
                            header_data_del.append(header.get_attribute("textContent").strip())
                        else:
                            continue
                    header_data_del = ['project_number'] + ['description'] + ['justification'] + ['advertise_date'] + ['bid_amount'] + ['bids_received_date'] + ['date_awarded'] + ['total_project_cost'] + ['first_chargeable_day'] + ['contractor'] + ['estimated_completion_date'] +['company_name']+['address']+['attorney_name']+['attorney_company_name'] + ['attorney_address'] + ['attorney_phone_num'] + ['bond_number'] + header_data_del+['Estimated_Cost_of_Final_Contract'] + ['Total_Estimate_to_Date'] + ['Contract_Award_Price'] + ['Auth_Retainage'] + ['Proposed_Time']+['Held_in_Securities'] + ['Time_Extended'] + ['Amt_to_be_Retained'] + ['Total_Time'] + ['Previous_Payment'] + ['Previous_time_charged']+['Liq_Damages_($/Day)'] + ['Time_Used_this_period'] + ['Time_Remaining'] + ['Misc_Deduction'] + ['Incentive_Prg(+/-)'] + ['Total_Deduction']+['Amount_this_Estimate'] # delete 'Amount_this_Estimate' later as it is same as 'Est Amt'
            except NoSuchElementException:
                print(value,"Out of scope/invalid format. Re-check periodically") 
                driver.get(url)
                continue
               
            # Extract Total Project Cost
            contract_info = []
            contract_info.append(value)
            for j, info in enumerate(info_table1):
                cells = info.find_elements(By.XPATH,'./td') 
                if j == 1 or j == 2:
                    contract_info.append(cells[1].get_attribute('textContent'))

            for c, row in enumerate(info_table2):
                cells = row.find_elements(By.XPATH,'./td') 
                if c == len(info_table2) - 1 or c == 1:
                    contract_info.append(cells[1].get_attribute('textContent'))
                else: 
                    contract_info.append(cells[1].get_attribute('textContent'))
                    contract_info.append(cells[3].get_attribute('textContent'))
            try:
                info_table_bond = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="bond-info"]/div/table/tbody/tr'))
                )
                for k, row in enumerate(info_table_bond):
                    cells = row.find_elements(By.XPATH,'./td')
                    if k == len(info_table_bond) - 1:
                        contract_info.append(extract_text(cells[1]))
                    else:
                        contract_info.append(extract_text(cells[1]))
                        contract_info.append(extract_text(cells[3]))
            except TimeoutException:
                 # Append empty values when bond information is not available to avoid assertion errors while creating data frames
                 for i in range(7):
                    contract_info.append(" ")    
                 print("No Bond Information",value)
            

            # if len(contract_info) != 17:
            #     print("check contract number data", value, i)
            time.sleep(5) # wait for transactions to load
            # Extract current amount
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//table[@ID="paymentInfo"]/tbody/tr'))
            )
            payment_data = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//table[@ID="paymentInfo"]/tbody/tr'))
            )
            for row in reversed(payment_data[-3:]): # Pull in the last 3 reports only
                current_row = []
                row_data = WebDriverWait(row, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, './td'))
                )
                if len(payment_data) <= 1 and len(row_data) <= 1:
                    current_row.extend(contract_info)
                    current_row.append("No Payment History Available")
                    row_data_list_del.append(current_row)
                    break

                current_row.extend(contract_info)
                # current_row.extend([cell.get_attribute("textContent") for cell in row_data if cell.get_attribute("textContent")])
                for i, data in enumerate(row_data):
                    if i == 1:
                        header_detail_link = data.find_element(By.XPATH,'./a')
                current_row.extend([cell.text for cell in row_data])
                del current_row[19:21]
                header_detail_link.send_keys(Keys.CONTROL + Keys.RETURN)
                
                # change the window of the driver since new tab is opened
                new_window_handle = [handle for handle in driver.window_handles if handle != original_window][-1]
                driver.switch_to.window(new_window_handle)
                time.sleep(0.5) # wait for page content to be loaded properly
                header_detail_info = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="project-info"]/div/table/tbody/tr'))
                )
                for k, detail in enumerate(header_detail_info):
                    cell_data = detail.find_elements(By.XPATH,'./td')
                    # Below conditions ensure inconsistent arrangement of data elements is handled
                    if k < 8 and k != 6:
                        # Extract only the values not the headers
                        current_row.append(cell_data[1].get_attribute('textContent'))
                        current_row.append(cell_data[3].get_attribute('textContent'))
                    elif k == 6:
                        current_row.append(str(cell_data[1].get_attribute('textContent')) + "-" + str(cell_data[2].get_attribute('textContent')))
                    else:
                        current_row.append(cell_data[3].get_attribute('textContent'))

                # Go back to details page
                # details_page = WebDriverWait(driver, 10).until(
                # EC.presence_of_element_located((By.LINK_TEXT, '[Back to Details Page]'))
                # )
                row_data_list_del.append(current_row)
                driver.close()
                driver.switch_to.window(original_window)
                # details_page.click() # go back to all estimates
                
                      
        except TimeoutException:        
            print([value,t],"Not Found")

        # except StaleElementReferenceException:
        #     print(value,"Loading Error")
                
        driver.get(url)

       
finally:
    # Close the browser
    driver.quit()
del_dot_data = pd.DataFrame(data=row_data_list_del, columns = header_data_del)


In [ ]:
# Email configuration
SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
SMTP_PORT = 587  # Usually 587 for TLS
EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECIPIENT
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email notification sent successfully.")
        logging.info("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")
        logging.error(f"Failed to send email notification: {e}")

# POST PROCESSING
df = del_dot_data.copy()
del df['Amount_this_Estimate']

df.rename(columns = {'project_number':'Project_Number','description':'Project_Description','justification':'Project_Justification','advertise_date':'Project_Advertise',
                     'bid_amount':'Project_Bid_Amount','bids_received_date':'Project_Bid_Rcvd','date_awarded':'Project_Bid_Awarded','total_project_cost':'Project_Cost_Total',
                     'first_chargeable_day':'Project_Start_Actual','contractor':'Contractor_Name','estimated_completion_date':'Project_Comp_Est','bond_number':'Bond_Number',
                     'company_name':'Bond_Company_Name','address':'Bond_Company_Address','attorney_name':'Bond_Attorney_Contact','attorney_company_name':'Bond_Attorney_Name',
                     'attorney_address':'Bond_Attorney_Address','attorney_phone_num':'Bond_Attorney_Phone','Est No':'Payment_Number','Status':'Payment_Status','Est Amt':'Payment_Amount',
                     'Payment Type':'Payment_Type','Advice No':'Payment_CheckNo','Advice Date':'Payment_Date','Total_Estimate_to_Date':'Payment_Amount_Total',
                     'Liq_Damages_($/Day)':'Liq_Damages_Per_Day','Incentive_Prg(+/-)':'Incentive_Prg_pos_or_neg'
                    }, inplace=True)

def parse_money(value):
    if not value:  # Check for None, empty string, or NaN
        return None
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value) if value.strip() else None

df['Project_Cost_Total'] = df['Project_Cost_Total'].apply(parse_money)
df['Payment_Amount_Total'] = df['Payment_Amount_Total'].apply(parse_money)
df['Project_Bid_Amount'] = df['Project_Bid_Amount'].apply(parse_money)
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)
df['Estimated_Cost_of_Final_Contract'] = df['Estimated_Cost_of_Final_Contract'].apply(parse_money)
df['Contract_Award_Price'] = df['Contract_Award_Price'].apply(parse_money)
df['Auth_Retainage'] = df['Auth_Retainage'].apply(parse_money)
df['Held_in_Securities'] = df['Held_in_Securities'].apply(parse_money)
df['Amt_to_be_Retained'] = df['Amt_to_be_Retained'].apply(parse_money)
df['Liq_Damages_Per_Day'] = df['Liq_Damages_Per_Day'].apply(parse_money)
df['Previous_Payment'] = df['Previous_Payment'].apply(parse_money)
df['Misc_Deduction'] = df['Misc_Deduction'].apply(parse_money)
df['Incentive_Prg_pos_or_neg'] = df['Incentive_Prg_pos_or_neg'].apply(parse_money)
df['Total_Deduction'] = df['Total_Deduction'].apply(parse_money)

df = df[df['Payment_Number'].str.isnumeric()]
df['Payment_Number'] = df['Payment_Number'].astype(int)
# Calculate Payment Balance
df['Payment_Balance'] = df['Project_Cost_Total'] - df['Payment_Amount_Total']

# Function to extract components
def extract_address_components(address):
    lines = address.split("\n")  # Split into lines
    
    if len(lines) < 2:
        return pd.NA, pd.NA, pd.NA, pd.NA  # Handle unexpected formats
    
    city_state_zip = lines[-1]  # Last line contains City, State, Zip
    parts = city_state_zip.split()  # Split by spaces

    if len(parts) < 3:  # Need at least City, State, Zip
        print(f"Malformed address: {address}")
        return pd.NA, pd.NA, pd.NA, pd.NA

    zipcode = parts[-1]  # Last part is ZIP code
    state = parts[-2]  # Second last is state
    city = " ".join(parts[:-2])  # Everything before state is city
    
    street = " ".join(lines[:-1])  # Everything before last line is street

    return street, city, state, zipcode

# Apply function to DataFrame
df[['Bond_Company_Street', 'Bond_Company_City', 'Bond_Company_State', 'Bond_Company_Zipcode']] = df['Bond_Company_Address'].apply(
    lambda x: pd.Series(extract_address_components(x))
)
df[['Bond_Attorney_Street', 'Bond_Attorney_City', 'Bond_Attorney_State', 'Bond_Attorney_Zipcode']] = df['Bond_Attorney_Address'].apply(
    lambda x: pd.Series(extract_address_components(x))
)

# Strip all leading and trailing whitespaces in all the string columns of a dataframe, before using drop_suplicates()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Delaware\data_store_DL.duckdb"
table_name = "DL_DOT"

# Current scraped data
scraped_data = df
print(df.columns)
print(len(df.columns))
# Connect to DuckDB
con = duckdb.connect(db_file)

# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Contractor_Name  TEXT,
Project_Number  TEXT,
Payment_Number  INTEGER,
Project_Description	TEXT,
Project_Justification  TEXT,
Project_Comp_Est  TEXT,
Project_Advertise  TEXT,
Project_Bid_Rcvd  TEXT,	
Project_Bid_Awarded  TEXT,	
Project_Start_Actual  TEXT,

Bond_Number  TEXT,
Bond_Company_Name  TEXT,
Bond_Company_Address  TEXT,
Bond_Attorney_Contact  TEXT,
Bond_Attorney_Name  TEXT,
Bond_Attorney_Address  TEXT,
Bond_Attorney_Phone  TEXT,
Bond_Company_Street  TEXT, 
Bond_Company_City  TEXT, 
Bond_Company_State  TEXT, 
Bond_Company_Zipcode  TEXT,
Bond_Attorney_Street  TEXT, 
Bond_Attorney_City  TEXT, 
Bond_Attorney_State  TEXT, 
Bond_Attorney_Zipcode  TEXT,

Project_Bid_Amount  DOUBLE,
Payment_Amount DOUBLE,
Payment_Status  TEXT,
Payment_Type  TEXT,
Payment_CheckNo  TEXT,
Payment_Date  TEXT,	
Project_Cost_Total  DOUBLE,	
Payment_Amount_Total  DOUBLE,
Payment_Total_Percent  FLOAT,
Payment_Balance  DOUBLE,
Pull_Date_Initial TEXT,
Payment_Amount_Percent FLOAT,

Estimated_Cost_of_Final_Contract  DOUBLE,
Contract_Award_Price  DOUBLE, 
Auth_Retainage  DOUBLE,
Proposed_Time  TEXT, 
Held_in_Securities  DOUBLE, 
Time_Extended  TEXT,
Amt_to_be_Retained  DOUBLE, 
Total_Time  TEXT, 
Previous_Payment  DOUBLE,
Previous_time_charged  TEXT, 
Liq_Damages_Per_Day  DOUBLE, 
Time_Used_this_period  TEXT,
Time_Remaining  TEXT, 
Misc_Deduction  DOUBLE, 
Incentive_Prg_pos_or_neg  DOUBLE,
Total_Deduction  DOUBLE
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()
print(existing_data.columns)
print(len(existing_data.columns))
# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial', 'Payment_Amount_Percent', 'Payment_Total_Percent'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Project_Number","Payment_Number", "Pull_Date_Initial"], ascending=[True,True,False])
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

else:
    combined_data = scraped_data
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    combined_data = combined_data.sort_values(by=["Project_Number", "Pull_Date_Initial"], ascending=[True,False])
    # Revert the formatting of pull_date_initial column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]

# Replace the table with the updated data
print(combined_data)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Close connection
con.close()
send_email("Python Script Execution Successful", "The job was executed successfully.")
print("Delaware scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'Delaware scraping completed and DUCKDB file updated Successfully.')

In [ ]:
# read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("Delaware", "data_store_DL", "DL_DOT")
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Delaware\Monthly\de_bulk_pipeline_april.xlsx")

LA City Pipeline 

In [ ]:
# Get the list of contract numbers with add info. Will be used as input in next step

# Automatic driver installer
service = Service(ChromeDriverManager().install())

# URL of the site
url = 'https://bca.lacity.gov/approvedsubs/'          

# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
row_data_list_la_sub = []
header_data_la_sub = ["work_order","project_title","subcontractor_name","work_description","work_value"]

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "company_wrapper"))
    )
    # Submit the search form
    try: 
        set_records = driver.find_element(By.XPATH, '//*[@id="company_length"]/label/select/option[4]')
        set_records.click()
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="company"]'))
        )
            
        # Iterate and extract work order no's & add info over all the pages
        next_page_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="company_next"]'))
            ) 
              
        while "disabled" not in next_page_button.get_attribute("class"):
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "company_wrapper"))
            )
            WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="company"]/tbody/tr'))
            )
            rows = driver.find_elements(By.XPATH, '//*[@id="company"]/tbody/tr')
            time.sleep(2)
            for i,row in enumerate(rows):
                row_data = row.find_elements(By.XPATH, './td')
                row_data_list_la_sub.append([cell.get_attribute("textContent") for cell in row_data[:-1]])   
            next_page_button_link = driver.find_element(By.XPATH,'//*[@id="company_next"]/a') 
            next_page_button_link.click()
            time.sleep(2)
            WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="company"]'))
            )             
            next_page_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="company_next"]'))
            )  
            if "disabled" in next_page_button.get_attribute("class"):
                rows =  WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="company"]/tbody/tr'))
                )
                for i,row in enumerate(rows):
                    row_data = driver.find_elements(By.XPATH, './td')
                    row_data_list_la_sub.extend([cell.get_attribute("textContent") for cell in row_data[:1]])

    except TimeoutException:
        print(value,"Not Found")
        

finally:
    # Close the browser
    driver.quit()
    
LA_city_data_sub = pd.DataFrame(data=row_data_list_la_sub, columns = header_data_la_sub)
def parse_money(value):
    if not value:  # Check for None, empty string, or NaN
        return None
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value) if value.strip() else None
# Convert 'work_value' to float for calculating 'Project_Cost_Total'
LA_city_data_sub['work_value'] = LA_city_data_sub['work_value'].apply(parse_money)
# Calculate 'Project_Cost_Total' by summing 'work_value'
LA_city_data_sub['Project_Cost_Total'] = LA_city_data_sub[LA_city_data_sub['work_value'] >= 0].groupby('work_order')['work_value'].transform('sum')


In [ ]:
# Get the associated data after extracting list of contract numbers in the previous step

# Automatic driver installer
service = Service(ChromeDriverManager().install())

# URL of the site
url = 'https://bca.lacity.org/Payments/payments_search.php'

# Contract number to search for
# contract_numbers= ['E1904038','SZC11765','SZC11922','E170128F','L0249155','E1904256',
#                     'SZC11321','E1700415','L0549492','EW40011F','E1907554','L0449473',
#                     'L1649430', 'E1907775','E1907703']
contract_numbers_list_la = LA_city_data_sub["work_order"].unique()
print("Total Contracts:",len(contract_numbers_list_la))
contract_numbers = contract_numbers_list_la

# Start a new browser session
driver = webdriver.Chrome(service=service)
driver.get(url)
row_data_list_la = []
header_data_la = []

try:
    for i,value in enumerate(contract_numbers):
    
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "query"))
        )

        # Find the contract number input box and enter the contract number
        contract_number_input = driver.find_element(By.NAME, "query")
        contract_number_input.clear()
        contract_number_input.send_keys(value)
        contract_number_input.send_keys(Keys.RETURN)
        # Submit the search form
        try: 
            
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'Payments_Search'))
            )
            

            table = driver.find_element(By.ID, 'Payments_Search').find_element(By.TAG_NAME, 'thead').find_element(By.TAG_NAME, 'tr')
            # Extract the table headers
            if i == 0: 
                headers = table.find_elements(By.TAG_NAME, 'th')
                header_data_la.extend([header.get_attribute("textContent").strip() for header in headers if header.get_attribute("textContent").strip()])
                
            if not header_data_la:
                headers = table.find_elements(By.TAG_NAME, 'th')
                header_data_la.extend([header.get_attribute("textContent").strip() for header in headers if header.get_attribute("textContent").strip()])
            set_records = driver.find_element(By.XPATH, '//*[@id="Payments_Search_length"]/label/select/option[4]') # Displays maximum number of records in a page for efficiency
            set_records.click()   
            # Iterate over all pages of transactions
            next_page_button = driver.find_element(By.XPATH,'//div[@id="Payments_Search_paginate"]/a[@id="Payments_Search_next"]')        
            while "disabled" not in next_page_button.get_attribute("class"):
                # Extract all records of transactions for a work order
                WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'Payments_Search'))
                )
                rows = driver.find_element(By.ID, 'Payments_Search').find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, 'td')  # Find data cells
                    row_data = [col.get_attribute("textContent")for col in cols]
                    if any(row_data):
                        row_data_list_la.append(row_data)
                next_page_button.click()
                WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'Payments_Search_paginate'))
                )             
                next_page_button = driver.find_element(By.XPATH,'//div[@id="Payments_Search_paginate"]/a[@id="Payments_Search_next"]') 
                
            # Last page needs to be extracted as well
            if "disabled" in next_page_button.get_attribute("class"):
                WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'Payments_Search'))
                )
                rows = driver.find_element(By.ID, 'Payments_Search').find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, 'td')  # Find data cells
                    row_data = [col.get_attribute("textContent") for col in cols]
                    if any(row_data):
                        row_data_list_la.append(row_data)
        
        except TimeoutException:
            print(value,"Data Not Found")
            
        driver.get(url) # Go back to search box
        

finally:
    # Close the browser
    driver.quit()
    
LA_city_data = pd.DataFrame(data=row_data_list_la, columns = header_data_la)


In [ ]:
# Prepare the main table using informaton from sub table
# Have only one instance of each work_order from the sub table to prepare for joining
LA_city_data_sub_unique = LA_city_data_sub.drop_duplicates(subset=LA_city_data_sub.loc[:, LA_city_data_sub.columns.isin(['work_order'])].columns,keep="first")
# Join the sub unique table with the main table in order to get 'project_title', 'Project_Cost_Total'
LA_city_data = pd.merge(LA_city_data, LA_city_data_sub_unique, left_on='Work Order', right_on='work_order',how='left')
# Drop unnecessary columns
LA_city_data.drop(columns=['subcontractor_name','work_description','work_value','work_order'], inplace=True)

In [ ]:
# Prepare the main table using informaton from sub table
def parse_money(value):
    if not value:  # Check for None, empty string, or NaN
        return None
    # Remove dollar signs and commas
    value = value.replace('$', '').replace(',', '')
    # Convert values in parentheses to negative numbers
    if '(' in value and ')' in value:
        value = '-' + value[1:-1]  # Remove the parentheses and add a negative sign
    return float(value) if value.strip() else None


# Email configuration
SMTP_SERVER = "smtp.office365.com"  # e.g., smtp.gmail.com
SMTP_PORT = 587  # Usually 587 for TLS
EMAIL_SENDER = "tarun.pongulaty@revealgc.com"
EMAIL_PASSWORD = ""  # Use app passwords or secure vaults
EMAIL_RECIPIENT = "tarun.pongulaty@revealgc.com"

def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = EMAIL_RECIPIENT
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email notification sent successfully.")
        logging.info("Email notification sent successfully.")
    except Exception as e:
        print(f"Failed to send email notification: {e}")
        logging.error(f"Failed to send email notification: {e}")

# Post Processing
df = LA_city_data.copy()
df_sub = LA_city_data_sub.copy()
df.rename(columns = {'Work Order':'Project_Number','project_title':'Project_Description_Full','Job Title':'Project_Description','Approved by Inspector':'Inspector_Approval',
                     'Approved by Supervisor':'Supervisor_Approval','Payment Number':'Payment_Number','Amount':'Payment_Amount','Received by Payments':'Payment_Received',
                     'Sent to Accounting':'Payment_toAccounting','Escrow Released':'Escrow_Released'}, inplace=True)
df_sub.rename(columns = {'work_order':'Project_Number','project_title':'Project_Description_Full','work_description':'Project_Justification_Sub'}, inplace=True)

# Convert data types of columns necessary for further calculations
df['Payment_Number'] = df['Payment_Number'].astype(int)                                                        
df['Payment_Amount'] = df['Payment_Amount'].apply(parse_money)
df['Escrow_Released'] = df['Escrow_Released'].apply(parse_money)

EST = pytz.timezone('US/Eastern')
now = datetime.now(EST)
current_date = now.strftime("%m/%d/%Y")
df["Pull_Date_Initial"] = current_date

# DUCKDB INTEGRATION
# File to store DuckDB data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\LA_City\data_store_LA.duckdb"
table_name = "LA_DOT"
table_name_sub = "LA_DOT_sub" 

# Current scraped data
scraped_data = df

# Connect to DuckDB
con = duckdb.connect(db_file)

# Create table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name} (
Project_Number  TEXT,
Payment_Number  INTEGER,
Project_Description_Full  TEXT,
Project_Description	TEXT,
Comments  TEXT,
Inspector_Approval  TEXT,
Supervisor_Approval  TEXT,

Payment_Received  TEXT,
Payment_toAccounting  TEXT,

Payment_Amount DOUBLE,	
Escrow_Released  DOUBLE,
Project_Cost_Total  DOUBLE,	
Payment_Amount_Total  DOUBLE,
Payment_Balance  DOUBLE,
Payment_Total_Percent  FLOAT,
Payment_Amount_Percent FLOAT,
Pull_Date_Initial TEXT
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data = con.execute(f"SELECT * FROM {table_name}").df()

# Deduplicate and merge
if not existing_data.empty:
    combined_data = pd.concat([existing_data, scraped_data], ignore_index=True)
    # find duplicates by all columns except the columns below since they are calculated later. Later, a logic can be developed to find revised payments if any
    combined_data = combined_data.drop_duplicates(subset=df.loc[:, ~df.columns.isin(['Pull_Date_Initial', 'Payment_Amount_Percent','Payment_Amount_Total','Payment_Total_Percent',
                                                                                     'Comments','Inspector_Approval','Supervisor_Approval','Payment_Balance'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    # Sort the DataFrame. Note: Sorting "Pull_Date_Initial" isn't required. Only if a payment number is duplicated for some reason.
    combined_data = combined_data.sort_values(by=["Project_Number", "Payment_Number","Pull_Date_Initial"], ascending=[True,True,False],na_position='first')
  
    # Calculate the Current Payment Amount Total
    combined_data['Payment_Amount_Total'] = combined_data[combined_data['Payment_Amount'] >= 0].groupby("Project_Number")['Payment_Amount'].cumsum()
    # Revert the formatting of the date column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment_Balance
    combined_data['Payment_Balance'] = combined_data['Project_Cost_Total'] - combined_data['Payment_Amount_Total']
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Calculate Payment Total Percent
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]   

# If loading data for the first time
else:
    combined_data = scraped_data
    # Post processing before loading into duckdb
    combined_data['Pull_Date_Initial'] = pd.to_datetime(combined_data['Pull_Date_Initial'])
    # Sort the DataFrame. Note: Sorting "Pull_Date_Initial" isn't required. Only if a payment number is duplicated for some reason.
    combined_data = combined_data.sort_values(by=["Project_Number", "Payment_Number","Pull_Date_Initial"], ascending=[True,True,False],na_position='first')
  
    # Calculate the Current Payment Amount Total
    combined_data['Payment_Amount_Total'] = combined_data[combined_data['Payment_Amount'] >= 0].groupby("Project_Number")['Payment_Amount'].cumsum()
    # Revert the formatting of the date column
    combined_data["Pull_Date_Initial"] = combined_data["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    # Calculate Payment_Balance
    combined_data['Payment_Balance'] = combined_data['Project_Cost_Total'] - combined_data['Payment_Amount_Total']
    # Calculate Payment Amount Percent
    combined_data['Payment_Amount_Percent'] = (combined_data['Payment_Amount']/combined_data['Payment_Amount_Total'] * 100).round(2)
    # Calculate Payment Total Percent
    combined_data['Payment_Total_Percent'] = (combined_data['Payment_Amount_Total']/combined_data['Project_Cost_Total'] * 100).round(2)
    table_info = con.execute(f"DESCRIBE {table_name}").fetchdf()
    correct_order = table_info['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data = combined_data[correct_order]  

# Current scraped_sub data
df_sub["Pull_Date_Initial"] = current_date
scraped_data_sub = df_sub


# Create sub_table if not exists
con.execute(f"""
CREATE TABLE IF NOT EXISTS {table_name_sub} (
Project_Number  TEXT,
Project_Description_Full  TEXT,
Project_Justification_Sub  TEXT,
subcontractor_name  TEXT,

work_value  DOUBLE,
Project_Cost_Total  DOUBLE,	
Pull_Date_Initial TEXT
)
""")

# Insert or Update Logic
# Load existing data from DuckDB
existing_data_sub = con.execute(f"SELECT * FROM {table_name_sub}").df()

# Deduplicate and merge
if not existing_data_sub.empty:
    combined_data_sub = pd.concat([existing_data_sub, scraped_data_sub], ignore_index=True)
    # find duplicates by all columns except Pull Date Initial
    combined_data_sub = combined_data_sub.drop_duplicates(subset=df_sub.loc[:, ~df_sub.columns.isin(['Pull_Date_Initial'])].columns,keep="first") 
    # Post processing before loading into duckdb
    combined_data_sub['Pull_Date_Initial'] = pd.to_datetime(combined_data_sub['Pull_Date_Initial'])
    # Sort the DataFrame. Note: Sorting "Pull_Date_Initial" isn't required. Only if a payment number is duplicated for some reason.
    combined_data_sub = combined_data_sub.sort_values(by=["Project_Number","Pull_Date_Initial"], ascending=[True,False])
    # Revert the formatting of the date column
    combined_data_sub["Pull_Date_Initial"] = combined_data_sub["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    table_info_sub = con.execute(f"DESCRIBE {table_name_sub}").fetchdf()
    correct_order_sub = table_info_sub['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data_sub = combined_data_sub[correct_order_sub]   

# If loading data for the first time
else:
    combined_data_sub = scraped_data_sub
   # Post processing before loading into duckdb
    combined_data_sub['Pull_Date_Initial'] = pd.to_datetime(combined_data_sub['Pull_Date_Initial'])
    # Sort the DataFrame. Note: Sorting "Pull_Date_Initial" isn't required. Only if a payment number is duplicated for some reason.
    combined_data_sub = combined_data_sub.sort_values(by=["Project_Number","Pull_Date_Initial"], ascending=[True,False])
    # Revert the formatting of the date column
    combined_data_sub["Pull_Date_Initial"] = combined_data_sub["Pull_Date_Initial"].dt.strftime('%m/%d/%Y')
    table_info_sub = con.execute(f"DESCRIBE {table_name_sub}").fetchdf()
    correct_order_sub = table_info_sub['column_name'].tolist()
    # Reorder the DataFrame to avoid conversion errors
    combined_data_sub = combined_data_sub[correct_order_sub] 

# Replace the table with the updated data
print(combined_data)
con.execute(f"DELETE FROM {table_name}")
con.execute(f"INSERT INTO {table_name} SELECT * FROM combined_data")

# Replace the sub_table with the updated data
con.execute(f"DELETE FROM {table_name_sub}")
con.execute(f"INSERT INTO {table_name_sub} SELECT * FROM combined_data_sub")

# Close connection
con.close()
send_email("Python Script Execution Successful", "The job was executed successfully.")
print("LA_City scraping completed and DUCKDB file updated Successfully.")
logging.info(
    'LA_City scraping completed and DUCKDB file updated Successfully.')

In [ ]:
#read data from corresponding duckdb table. Specify State, file and table name respectively
data = get_data("LA_City", "data_store_LA", "LA_DOT")

In [ ]:
data.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\LA_City\Monthly\la_city_pipeline_april.xlsx")

In [ ]:
data_sub = get_data("LA_City", "data_store_LA", "LA_DOT_sub")

In [ ]:
data_sub.to_excel(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\LA_City\Monthly\la_city_pipeline_sub_april.xlsx")

Examples on Alterations to existing DuckDB

In [ ]:
# For renaming columns
import duckdb
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Oklahoma\data_store_ok.duckdb"
# Create an in-memory DuckDB database and connection
con = duckdb.connect(db_file)
# Rename columns
con.execute("ALTER TABLE OK_DOT RENAME COLUMN bid_amount TO Project_Cost_Total;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN funds_available_bid_co TO Project_Cost_Adjusted;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN completion_date TO Project_Comp_Final;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN date_time_stopped TO Project_Charges_Stopped;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN surety_company TO Bond_Company_Name;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN prime_cont TO Contractor_Name;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN this_estimate TO Payment_Amount;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN date_approved TO Payment_Approved;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN total_to_date TO Payment_Amount_Total;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN unearned_balance TO Payment_Balance;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN percent_complete TO Payment_Total_Percent;")

# Verify the changes
print(con.execute("DESCRIBE OK_DOT").fetchall())
con.close()

In [ ]:
# For dropping columns
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Washington\data_store_wa.duckdb"
con = duckdb.connect(db_file)
# # Delete a column
con.execute("ALTER TABLE WA_DOT DROP COLUMN Project_Total_Percent;")

# Verify the changes
print(con.execute("DESCRIBE WA_DOT").fetchall())
con.close()

In [ ]:
# For updating existing columns in case of any changes in transformation methods
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Illinois\data_store_IL.duckdb"
con = duckdb.connect(db_file)
# Update the column with the formula
con.execute("""
    UPDATE IL_DOT 
    SET Payment_Balance = ROUND(Project_Cost_Total_Adjusted - Payment_Amount_Total, 2);
""")
# Verify the results
print(con.execute("SELECT * FROM IL_DOT").fetchall())
con.close()

In [ ]:
# Change the data types of existing columns in duckdb
# Step 3: Add new column with proper type and data
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Oklahoma\data_store_ok.duckdb"
con = duckdb.connect(db_file)
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN org_cont_time_double DOUBLE;
""")
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN current_time_allowed_double DOUBLE;     
""")

con.execute("""
    UPDATE OK_DOT
    SET
        org_cont_time_double = TRY_CAST(org_cont_time AS DOUBLE),
        current_time_allowed_double = TRY_CAST(current_time_allowed AS DOUBLE);
""")

# Step 4: Drop the original column
con.execute("ALTER TABLE OK_DOT DROP COLUMN org_cont_time;")
con.execute("ALTER TABLE OK_DOT DROP COLUMN current_time_allowed;")

# Step 5: Rename the new column to the original name
con.execute("ALTER TABLE OK_DOT RENAME COLUMN org_cont_time_double TO org_cont_time;")
con.execute("ALTER TABLE OK_DOT RENAME COLUMN current_time_allowed_double TO current_time_allowed;")

# Step 6: Check the result
df = con.execute("SELECT * FROM OK_DOT;").fetchdf()
print(df.head())
con.close()

In [ ]:
# For creating new derived columns
db_file = r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Oklahoma\data_store_ok.duckdb"
con = duckdb.connect(db_file)
# 1. Add the new columns
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN Project_Cost_Additions DOUBLE;
""")
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN Payment_Amount_Percent DOUBLE;
""")
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN Payment_Date TEXT;
""")
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN Project_Comp_Est TEXT;
""")
con.execute("""
    ALTER TABLE OK_DOT ADD COLUMN Project_Comp_Substantial TEXT;
""")

# 2. Update values using formulas
con.execute("""
    UPDATE OK_DOT
    SET 
        Project_Cost_Additions = TRY_CAST((Project_Cost_Adjusted - Project_Cost_Total) AS DOUBLE),
        Payment_Amount_Percent = TRY_CAST(ROUND((Payment_Amount / Payment_Amount_Total) * 100, 2) AS DOUBLE),
        Payment_Date = SUBSTR(Payment_Work_Period, LENGTH(Payment_Work_Period) - 9, 10),
        Project_Comp_Est = STRFTIME(
            STRPTIME(date_ntp_issued, '%m/%d/%Y') + (CAST(org_cont_time AS INTEGER) || ' days')::INTERVAL,
            '%m/%d/%Y'
        ),
        Project_Comp_Substantial = STRFTIME(
            STRPTIME(date_ntp_issued, '%m/%d/%Y') + (CAST(current_time_allowed AS INTEGER) || ' days')::INTERVAL,
            '%m/%d/%Y'
        );
""")

# 3. View result
df = con.execute("SELECT * FROM OK_DOT;").fetchdf()
df.head()


MFT Function

In [ ]:
""" 
# census_mft_resource.py
This script transfers files to census via MFT
"""

import base64
import re
import subprocess
from typing import Optional
from pathlib import Path


class MFTClient:
    """
    A client for sending files to a Managed File Transfer (MFT) server using cURL.

    Attributes:
        url (str): The base URL of the MFT server.
        username (str): The username for authentication.
        password (str): The password for authentication.

    Methods:
        send_file(target_file: Path, dest_name: str, dest_folder="") -> subprocess.CompletedProcess:
            Uploads a file to the specified MFT destination folder.
    """

    def __init__(
        self,
        url: str,
        token: Optional[str] = None,
        username: Optional[str] = None,
        password: Optional[str] = None,
    ):
        """
        Initializes the MFTClient with server credentials.

        Args:
            url (str): The base URL of the MFT server.
            username (str): The username for authentication.
            password (str): The password for authentication.
        """
        self.url = url
        if token is None and (username is None or password is None):
            raise ValueError("Either token or username and password must be provided.")
        if token is not None and (username is not None or password is not None):
            raise ValueError(
                "Only one of token or username and password can be provided."
            )

        self.token = (
            token if token is not None else self._encode_credentials(username, password)
        )

    def _encode_credentials(self, username, password) -> str:
        """
        Encodes the username and password for use in HTTP Basic Authentication.

        Returns:
            str: The encoded credentials.
        """
        return base64.b64encode(f"{username}:{password}".encode()).decode()

    def send_file(self, target_file: Path, dest_name: str, dest_folder=""):
        """
        Uploads a file to the MFT server.

        Args:
            target_file (Path): The local file path to upload.
            dest_name (str): The name the file should have on the MFT server.
            dest_folder (str, optional): The destination folder on the MFT server. Defaults to "".

        Returns:
            subprocess.CompletedProcess: The result of the cURL command execution.

        Raises:
            FileNotFoundError: If the specified file does not exist.
            ValueError: If authentication credentials are missing.
        """

        # create user token
        file_path = Path(target_file).resolve()
        if not file_path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")

        folder_url = self.url + dest_folder

        # verify file exists

        # create curl command
        mft_str = (
            f'curl -X POST "{folder_url}/{dest_name}?packet=1&position=0&final=true" '
            f'-H "Authorization: Basic {self.token}" '
            f'-T "{target_file}" -v'
        )

        try:
            # send file wit subprocess logging
            result = subprocess.run(
                mft_str, shell=True, capture_output=True, text=True, check=True
            )
            success = True

        except subprocess.CalledProcessError as e:
            result = e
            success = False

        redacted_stdout = self._scrub_sensitive_data(result.stdout)
        redacted_stderr = self._scrub_sensitive_data(result.stderr)

        return {
            "stdout": redacted_stdout,
            "stderr": redacted_stderr,
            "returncode": result.returncode,
            "error": None if success else "File transfer failed.",
        }

    def _scrub_sensitive_data(self, text):
        """
        Removes sensitive information from subprocess output before logging.

        Args:
            text (str): The original stdout or stderr text.

        Returns:
            str: The redacted text with authentication details removed.
        """
        if not text:
            return "No output"

        # Use regex to remove the Authorization header details
        redacted_text = re.sub(
            r"Authorization: Basic [A-Za-z0-9+/=]+",
            "Authorization: Basic ***REDACTED***",
            text,
        )

        return redacted_text


# Example usage of MFTClient
if __name__ == "__main__":
    import os

    # Load environment variables for credentials (or set manually)
    MFT_URL = os.getenv(
        "MFT_BASE_URL", "https://mft.econ.census.gov/cfcc/rest/ft/v3/transfer/"
    )
    # MFT_USERNAME = os.getenv("your_username", default="MFT_USERNAME")
    MFT_USERNAME = ""
    # MFT_PASSWORD = os.getenv("your_password", default="MFT_PASSWORD")
    MFT_PASSWORD = ""

    # Create an MFTClient instance
    mft_client = MFTClient(url=MFT_URL, username=MFT_USERNAME, password=MFT_PASSWORD)

    # Define file to upload

    try:
        # Upload file to MFT server
        res = mft_client.send_file(
            Path(r"C:\Users\TarunPongulaty\Documents\Revealgc\Reveal_Census - databases\Tarun\dot_scraping\Wisconsin\wis_dot_api_test8.xlsx"), "vip_wisconsin_test", "rgc_analyzed"
            )

        # Print upload response
        print("STDOUT:", res["stdout"])
        print("STDERR:", res["stderr"])

    except FileNotFoundError as e:
        print("File not found error:", e)
    except subprocess.CalledProcessError as e:
        print("Subprocess error:", e)
